# Predictive Maintenance mit SCANIA-Daten – Data Preperation

**Projekt:** Bachelorarbeit Data Science  
**Thema:** 
**Datengrundlage:** SCANIA Component X Dataset  
**Autor:** Justin Stange-Heiduk  
**Betreuung:** Dr. Martin Prause  
**Ziel:** Erstellen und testen der Daten Vorbereitung Funktionen  

---

**Erstellt:** 2025-08-05  
**Letzte Änderung:** 2025-09-1


---

### 1. Laden der Daten
### 2. Interpolation fehlender Werte
### 3. Differenzbildung bei kumulativen Sensorwerten
### 4. Rolling Windows
### 5. Feature-Extraktion mit tsfresh
### 6. Label-Erstellung
### 7. Feature-Auswahl

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.feature_calculators import *
from tsfresh.feature_extraction.settings import from_columns
from tsfresh.utilities.distribution import MultiprocessingDistributor
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test
import gc
from typing import Dict, Union
import json
from pathlib import Path
import os


In [2]:
%run CommonFunctions.ipynb

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

### 1. raw_date laden

In [ ]:
raw_data = load_all_raw_data()

## Data Preperation for Survivial Analyse

### 2. Interpolation fehlender Werte

In [ ]:
for key in raw_data.keys():
    locals()[f"readouts_{key}"] = raw_data[key]["readouts"]

In [ ]:
def interpolate_readout_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Performs linear interpolation of missing values in time series sensor data per vehicle.

    The function:
    - Sorts the input DataFrame by 'vehicle_id' and 'time_step'
    - Applies linear interpolation separately for each vehicle
    - Interpolates all numeric feature columns except 'vehicle_id' and 'time_step'
    - Fills missing values in both directions (forward and backward)

    Args:
        df (pd.DataFrame): The raw readout DataFrame containing sensor values, 
            including the columns 'vehicle_id' and 'time_step'

    Returns:
        pd.DataFrame: A new DataFrame with interpolated sensor features. 
            Original index is not preserved. Interpolation is applied only to 
            feature columns; identifiers remain unchanged.
    """
    # Sort by vehicle_id and time_step
    df = df.sort_values(by=["vehicle_id", "time_step"])

    # Interpolate numeric columns, excluding 'vehicle_id' and 'time_step'
    feature_cols = df.select_dtypes(include=["number"]).columns.difference(["vehicle_id", "time_step"])
    df[feature_cols] = df.groupby("vehicle_id")[feature_cols].transform(lambda x: x.interpolate(method="linear", limit_direction="both"))

    # Reset index to avoid keeping the original index
    return df.reset_index(drop=True)

In [ ]:
for key in raw_data.keys():
    locals()[f"readouts_inter_{key}"] = interpolate_readout_df(raw_data[key]["readouts"])

In [ ]:
def interpolate_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Interpolate missing values in the DataFrame

    Args:
        df (pd.DataFrame): The input raw_data with potential missing values.

    """
    
    for key in raw_data.keys():
        locals()[f"readouts_inter_{key}"] = interpolate_readout_df(raw_data[key]["readouts"])
    


In [ ]:
feature_cols = readouts_train.select_dtypes(include=["number"]).columns.difference(["vehicle_id", "time_step"])

original_stats_train = readouts_train[feature_cols].describe()
interp_stats_train = readouts_inter_train[feature_cols].describe()

original_stats_test = readouts_test[feature_cols].describe()
interp_stats_test = readouts_inter_test[feature_cols].describe()

original_stats_validation = readouts_validation[feature_cols].describe()
interp_stats_validation = readouts_inter_validation[feature_cols].describe()

print("Stats (train):")
display(original_stats_train/interp_stats_train)
print("Stats (test):")
display(original_stats_test/interp_stats_test)
print("Stats (validation):")
display(original_stats_validation/interp_stats_validation)

In [ ]:
for split in raw_data.keys():
        df = raw_data[split]['readouts']
        locals()[f"{split}_missing_values"] = df.isnull().sum().sum()
        

readouts_inter_train_missing_values = readouts_inter_train.isnull().sum().sum()
readouts_inter_test_missing_values = readouts_inter_test.isnull().sum().sum()
readouts_inter_validation_missing_values = readouts_inter_validation.isnull().sum().sum()

print(f"Missing values in train: before={train_missing_values}, after={readouts_inter_train_missing_values}")
print(f"Missing values in test: before={test_missing_values}, after={readouts_inter_test_missing_values}")
print(f"Missing values in validation: before={validation_missing_values}, after={readouts_inter_validation_missing_values}")


In [ ]:
missing_columns = readouts_inter_train.loc[:, readouts_inter_train.isna().any()].columns.tolist()

display(readouts_inter_train[readouts_inter_train[missing_columns].isna().any(axis=1)])

In [ ]:
missing_columns = readouts_inter_test.loc[:, readouts_inter_test.isna().any()].columns.tolist()
display(readouts_inter_test[readouts_inter_test[missing_columns].isna().any(axis=1)])

#### Remove vehicles with nan Value

In [ ]:
def remove_vehicles_with_nans_from_split(
    readouts_df: pd.DataFrame,
    raw_split_data: dict,
    split_name: str
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Removes vehicles with remaining NaN values from readouts, labels, and specs 
    for a given dataset split (train, test, validation).

    Args:
        readouts_df (pd.DataFrame): Interpolated readouts DataFrame.
        raw_split_data (dict): Dictionary containing 'tte' or 'labels' and 'spec' DataFrames.
        split_name (str): Name of the data split ("train", "test", or "validation").

    Returns:
        tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: Filtered readouts, labels/tte, and specs.
    """
    # NaN-Spalten identifizieren
    missing_cols = readouts_df.columns[readouts_df.isna().any()].tolist()

    if not missing_cols:
        print(f"[{split_name}] Keine fehlenden Werte – keine Fahrzeuge entfernt.")
        label_df = raw_split_data["tte"] if split_name == "train" else raw_split_data["labels"]
        return readouts_df.copy(), label_df.copy(), raw_split_data["spec"].copy()

    # Fahrzeuge mit NaNs finden
    vehicles_with_nans = (
        readouts_df.loc[readouts_df[missing_cols].isna().any(axis=1), "vehicle_id"]
        .unique()
    )

    print(f"[{split_name}] {len(vehicles_with_nans)} Fahrzeug(e) mit NaNs entfernt.")
    print(f"   Entfernte vehicle_ids: {list(vehicles_with_nans)}")

    # Entsprechende DataFrames bereinigen
    readouts_clean = readouts_df[~readouts_df["vehicle_id"].isin(vehicles_with_nans)].copy()

    label_key = "tte" if split_name == "train" else "labels"
    labels_clean = raw_split_data[label_key][
        ~raw_split_data[label_key]["vehicle_id"].isin(vehicles_with_nans)
    ].copy()

    specs_clean = raw_split_data["spec"][
        ~raw_split_data["spec"]["vehicle_id"].isin(vehicles_with_nans)
    ].copy()

    return readouts_clean, labels_clean, specs_clean


In [ ]:
readouts_cleaned_train, tte_train_df, specs_train_df = remove_vehicles_with_nans_from_split(
    readouts_inter_train, raw_data["train"], "train"
)

readouts_cleaned_test, tte_test_df, specs_test_df = remove_vehicles_with_nans_from_split(
    readouts_inter_test, raw_data["test"], "test"
)

readouts_cleaned_validation, tte_val_df, specs_val_df = remove_vehicles_with_nans_from_split(
    readouts_inter_validation, raw_data["validation"], "validation"
)


In [ ]:
readouts_inter_train_missing_values = readouts_cleaned_train.isnull().sum().sum()
readouts_inter_test_missing_values = readouts_cleaned_test.isnull().sum().sum()
readouts_inter_validation_missing_values = readouts_cleaned_validation.isnull().sum().sum()

print(f"Missing values in train: before={train_missing_values}, after={readouts_inter_train_missing_values}")
print(f"Missing values in test: before={test_missing_values}, after={readouts_inter_test_missing_values}")
print(f"Missing values in validation: before={validation_missing_values}, after={readouts_inter_validation_missing_values}")


### 3. Differenzbildung bei kumulativen Sensorwerten

In [ ]:
def compute_differences_per_vehicle_test(readouts_df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes first-order differences for cumulative sensor features per vehicle.
    For the first time step, uses the original value instead of NaN or 0.
    Test with columns _diff for each sensor bin.

    Args:
        readouts_df (pd.DataFrame): Readouts DataFrame with 'vehicle_id' and 'time_step'.
        
    Returns:
        pd.DataFrame: DataFrame with additional '_diff' columns per selected sensor bin.
    """
    df = readouts_df.copy()
    
    
    bin_cols = [col for col in df.columns.difference(["vehicle_id", "time_step"])]

    for col in bin_cols:
        new_col = f"{col}_diff"

        # Berechne Differenz pro vehicle_id
        df[new_col] = df.groupby("vehicle_id")[col].diff()

        # Ersetze NaN im ersten Schritt durch den Originalwert
        first_values = df.groupby("vehicle_id")[col].transform("first")
        df[new_col] = df[new_col].fillna(first_values)

    return df


In [ ]:
readouts_diff_train = compute_differences_per_vehicle_test(readouts_cleaned_train)

In [ ]:
display(readouts_diff_train[readouts_diff_train["vehicle_id"] == 5][["427_0", "427_0_diff"]])

In [ ]:
def compute_differences_per_vehicle(readouts_df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes first-order differences for cumulative sensor features per vehicle.
    For the first time step, uses the original value instead of NaN or 0.
    

    Args:
        readouts_df (pd.DataFrame): Readouts DataFrame with 'vehicle_id' and 'time_step'.
        
    Returns:
        pd.DataFrame: DataFrame with additional '_diff' columns per selected sensor bin.
    """
    df = readouts_df.copy()
    
    
    bin_cols = [col for col in df.columns.difference(["vehicle_id", "time_step"])]

    for col in bin_cols:
        
                
        first_values = df.groupby("vehicle_id")[col].transform("first")
        # Berechne Differenz pro vehicle_id
        df[col] = df.groupby("vehicle_id")[col].diff()

        # Ersetze NaN im ersten Schritt durch den Originalwert
        df[col] = df[col].fillna(first_values)

    return df


In [ ]:
readouts_diff_train = compute_differences_per_vehicle(readouts_cleaned_train)
readouts_diff_test = compute_differences_per_vehicle(readouts_cleaned_test)
readouts_diff_validation = compute_differences_per_vehicle(readouts_cleaned_validation)

In [ ]:
save_df(readouts_diff_train, ordner="02_intermediate", name="readouts_diff_train")
save_df(readouts_diff_test, ordner="02_intermediate", name="readouts_diff_test")
save_df(readouts_diff_validation, ordner="02_intermediate", name="readouts_diff_validation")

NameError: name 'readouts_diff_test' is not defined

In [5]:
# readouts_diff_train = load_df(ordner="02_intermediate", name="readouts_diff_train")
readouts_diff_test = load_df(ordner="02_intermediate", name="readouts_diff_test")
# readouts_diff_validation = load_df(ordner="02_intermediate", name="readouts_diff_validation")

In [103]:
readouts_diff_validation = load_df(ordner="02_intermediate", name="readouts_diff_validation")

In [26]:
display(readouts_diff_test.head())

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,167_4,167_5,167_6,167_7,167_8,167_9,309_0,272_0,272_1,272_2,272_3,272_4,272_5,272_6,272_7,272_8,272_9,835_0,370_0,291_0,291_1,291_2,291_3,291_4,291_5,291_6,291_7,291_8,291_9,291_10,158_0,158_1,158_2,158_3,158_4,158_5,158_6,158_7,158_8,158_9,100_0,459_0,459_1,459_2,459_3,459_4,459_5,459_6,459_7,459_8,459_9,459_10,459_11,459_12,459_13,459_14,459_15,459_16,459_17,459_18,459_19,397_0,397_1,397_2,397_3,397_4,397_5,397_6,397_7,397_8,397_9,397_10,397_11,397_12,397_13,397_14,397_15,397_16,397_17,397_18,397_19,397_20,397_21,397_22,397_23,397_24,397_25,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,1,4.4,94410.0,7982.0,3115020.0,2625.0,1304.0,366624.0,232040.0,147104.0,422728.0,1651432.0,223425.0,55640.0,8816.0,3449.0,35.0,589556.0,165737.0,107063.0,255684.0,3008236.0,173097.0,6589.0,0.0,0.0,0.0,4466869.0,23994.0,328.0,184.0,237.0,329.0,104.0,48.0,84.0,76.0,60.0,144.0,28.0,132118.0,1565102.0,1976016.0,469225.0,314398.0,312200.0,194684.0,135352.0,150738.0,230483.0,232410.0,199.198833,54.364944,45.376778,51.823889,63.903611,77.826611,103.357333,122.073778,445.946667,548.500944,347.169167,372.139389,386.215167,499.586222,801.094944,3222.756944,2.499500,0.111111,0.019500,0.000000,263249.0,239652.0,280203.0,26432.0,21462.0,2674.0,409690.0,144340.0,519841.0,5432.0,1408.0,77.0,53152.0,67929.0,242384.0,5194.0,1653.0,63.0,8961.0,41455.0,127793.0,6974.0,952.0,14.0,5489.0,29540.0,72548.0,9696.0,1309.0,7.0,399.0,31843.0,89138.0,24101.0,5112.0,7.0
1,1,5.0,10995.0,1196.0,352220.0,600.0,1.0,24272.0,20488.0,10720.0,26761.0,205368.0,30168.0,12352.0,2232.0,0.0,25.0,33452.0,20141.0,11066.0,11605.0,353243.0,29744.0,0.0,0.0,0.0,0.0,476832.0,1899.0,14.0,12.0,14.0,16.0,9.0,0.0,16.0,16.0,2.0,18.0,2.0,25970.0,151032.0,151480.0,42910.0,41188.0,53354.0,34790.0,19334.0,22652.0,42112.0,16855.0,15.360611,3.551833,4.019444,4.591333,7.239278,8.834500,15.243556,22.044667,58.584056,60.476056,41.487333,45.604333,47.604278,81.691278,94.827500,457.780167,0.854333,0.045944,0.002111,0.005111,30016.0,25046.0,42129.0,4692.0,4866.0,791.0,27671.0,8115.0,28063.0,407.0,161.0,28.0,4040.0,6295.0,35646.0,672.0,281.0,7.0,752.0,4970.0,22695.0,932.0,161.0,0.0,688.0,4229.0,10599.0,2346.0,532.0,0.0,56.0,3906.0,17228.0,4067.0,308.0,0.0
2,1,8.2,109035.0,14170.0,3618550.0,4785.0,3592.0,70344.0,125736.0,155192.0,348912.0,1923624.0,545904.0,95384.0,9392.0,1568.0,20.0,143452.0,115918.0,114554.0,275550.0,3529889.0,351791.0,6083.0,0.0,0.0,0.0,4711729.0,45207.0,130.0,84.0,136.0,246.0,69.0,30.0,107.0,80.0,22.0,168.0,42.0,188454.0,1158864.0,1804796.0,569394.0,466060.0,363244.0,258580.0,204386.0,256116.0,504770.0,138354.0,104.317944,36.328889,38.761167,48.907000,63.916778,81.614556,123.079444,167.179889,688.146167,885.584722,561.994056,735.728611,678.722722,765.459167,1277.286444,3664.576000,4.009444,0.181833,0.012722,0.000000,300644.0,292754.0,419727.0,46873.0,42980.0,6671.0,188693.0,129928.0,283080.0,5454.0,1610.0,231.0,19047.0,90503.0,308315.0,7134.0,2122.0,98.0,6748.0,60984.0,168652.0,8828.0,1589.0,63.0,6545.0,41678.0,114943.0,14057.0,2675.0,14.0,665.0,58268.0,187096.0,57428.0,11235.0,7.0
3,1,9.8,45750.0,5772.0,1611060.0,4515.0,1616.0,32488.0,77088.0,50968.0,166512.0,772576.0,308544.0,81320.0,10064.0,6008.0,140.0,45056.0,88297.0,35684.0,148709.0,1520871.0,236071.0,15213.0,0.0,0.0,0.0,2169949.0,16569.0,101.0,76.0,96.0,120.0,36.0,40.0,46.0,28.0,35.0,114.0,12.0,87346.0,732466.0,796852.0,215503.0,179508.0,135394.0,103726.0,83594.0,114058.0,211106.0,82053.0,51.635111,23.030667,19.974167,24.711500,31.951611,48.177944,63.630444,85.909722,336.708444,496.825444,323.425111,375.903944,340.031000,308.924889,447.205389,1238.801778,1.587500,0.032944,0.026167,0.000000,221039.0,173313.0,196364.0,23870.0,20471.0,3136.0,43484.0,74487.0,87389.0,3354.0,1547.0,126.0,7077.0,44625.0,97034.0,3738.0,1521.0,84.0,3304.0,29254.0,67040.0,4483.0,771.0,14.0,2667.0,19204.0,46236.0,6609.0,1198.0,28.0,378.0,24332.0,84714.0,29534.0,7399.

In [ ]:
display(readouts_diff_train[readouts_diff_train["vehicle_id"] == 0]["158_1"])

### 4. Rolling Windows

In [7]:
def create_all_fixed_time_index_windows_faster(
    readouts_df: pd.DataFrame,
    window_sizes: list[float]
) -> pd.DataFrame:
    base_cols = ["vehicle_id", "time_step"]
    sensor_cols = [c for c in readouts_df.columns if c not in base_cols]

    # Melt einmalig in long-Format
    melted = readouts_df.melt(
        id_vars=base_cols,
        value_vars=sensor_cols,
        var_name="kind",
        value_name="value"
    )

    # Optimierte dtypes
    melted = melted.astype({
        "vehicle_id": "uint16",          # reicht völlig für ≤ 65535 Fahrzeuge
        "time_step": "float64",
        "kind": "category",              # viele Wiederholungen → spart RAM
        "value": "float32",              # falls hohe Präzision nicht nötig
    }, copy=False)

    melted = melted.sort_values(["vehicle_id", "time_step"], kind="mergesort", ignore_index=True)
    print(f"anzhal an vehicle ids: {melted['vehicle_id'].nunique()}")

    # Gruppieren (RAM-schonend) – dict bleibt
    vehicle_groups = {vid: g for vid, g in melted.groupby("vehicle_id", sort=False)}
    n_vehicles = len(vehicle_groups)

    out_parts: list[pd.DataFrame] = []

    for w in window_sizes:
        print(f"Windows sized {w} wird gemacht")

        for vid, g in tqdm(
            vehicle_groups.items(),
            total=n_vehicles,
            desc=f"Vehicles (w={w})",
            leave=True
        ):
            t = g["time_step"].to_numpy()
            uniq_t = np.unique(t)
            if uniq_t.size == 0:
                continue

            starts = np.searchsorted(t, uniq_t - w, side="right")
            ends   = np.searchsorted(t, uniq_t,     side="left")

            def _fmt_ts(x: float) -> str:
                s = f"{x:.1f}"
                return s.rstrip("0").rstrip(".") if "." in s else str(x)

            # Optional: Nur bei Bedarf!
            id_suffixes = [f"w{w}__id{vid}_t{_fmt_ts(ct)}" for ct in uniq_t]

            for i_start, i_end, win_id, ct in zip(starts, ends, id_suffixes, uniq_t):
                if i_end <= i_start:
                    # Fallback: nimm nur den letzten Wert als 1-Zeilen-Window
                    fallback_time = t[i_end - 1]
                    fallback_rows = g[g["time_step"] == fallback_time].copy()
                    fallback_rows["id"] = win_id
                    fallback_rows["time_step_current"] = np.float64(ct)
                    out_parts.append(fallback_rows)
                    continue
                win_slice = g.iloc[i_start:i_end].copy()

                win_slice["id"] = win_id
                win_slice["time_step_current"] = np.float64(ct)

                out_parts.append(win_slice)

    if not out_parts:
        return pd.DataFrame(columns=["id", "vehicle_id", "time_step", "time_step_current", "kind", "value"])

    final = pd.concat(out_parts, ignore_index=True)

    final = final.astype({
        "vehicle_id": "uint16",
        "time_step": "float64",
        "time_step_current": "float64",
        "kind": "category",
        "value": "float32",
        "id": "object",  # hier keine Alternative, da unique string – ggf. später entfernen
    }, copy=False)


    print(f"anzhal an vehicle ids: {final['vehicle_id'].nunique()}")
    # Alle float32-Spalten auf 1 Nachkommastelle runden
    float_cols = final.select_dtypes(include=["float32"]).columns
    final[float_cols] = final[float_cols].round(1)


    return final[["id", "vehicle_id", "time_step", "time_step_current", "kind", "value"]]


In [ ]:
windowed_df = create_all_fixed_time_index_windows_faster(
    readouts_df=readouts_diff_train[readouts_diff_train["vehicle_id"] == 0],
    window_sizes=[8],
)

display(windowed_df.kind.unique().shape)
display(windowed_df[windowed_df["vehicle_id"] == 0][windowed_df["kind"] == "158_1"].head(50))
display(readouts_diff_train[readouts_diff_train["vehicle_id"] == 0][["time_step", "158_1"]].head(50))

In [8]:
combined_window_df_8_train = create_all_fixed_time_index_windows_faster(
    readouts_df=readouts_diff_train,
    window_sizes=[8]
)

anzhal an vehicle ids: 23548
Windows sized 8 wird gemacht


Vehicles (w=8):   0%|          | 0/23548 [00:00<?, ?it/s]

anzhal an vehicle ids: 23548


### Für C-Index Test Daten

In [ ]:
combined_window_df_8_test_all = create_all_fixed_time_index_windows_faster(
    readouts_df=readouts_diff_test,
    window_sizes=[8]
)

anzhal an vehicle ids: 5043
Windows sized 8 wird gemacht


Vehicles (w=8):   0%|          | 0/5043 [00:00<?, ?it/s]

anzhal an vehicle ids: 5043


In [9]:
save_df(df = combined_window_df_8_train, ordner="03_primary", name = "combined_window_df_8_train")


In [ ]:
save_df(df = combined_window_df_8_test_all, ordner="03_primary", name = "combined_window_df_8_train_all")

In [7]:
combined_window_df_8_train = load_df(ordner="03_primary", name = "combined_window_df_8_train")

In [12]:
gc.collect()

0

In [4]:
combined_window_df_8_test_all = load_df(ordner="03_primary", name = "combined_window_df_8_train_all")

In [11]:
combined_window_df_8_test_all.vehicle_id.shape

(37845150,)

In [114]:
def last_readout_window_per_window(df: pd.DataFrame, window_size: int) -> pd.DataFrame:
    """
    Funktion, um das letzte Readout innerhalb eines bestimmten Zeitfensters für jedes Fahrzeug zu ermitteln.

    Args:
        df (pd.DataFrame): Das Eingabe-DataFrame mit Fahrzeugdaten.
        window_size (int): Die Größe des Zeitfensters.

    Returns:
        pd.DataFrame: Ein DataFrame mit den letzten Readouts innerhalb des Zeitfensters für jedes Fahrzeug.
    """

    # Schritt 1: Max time_step je Fahrzeug
    last_steps = (
        df
        .groupby("vehicle_id", sort=False)["time_step"]
        .max()
        .reset_index()
        .rename(columns={"time_step": "last_time"})
    )

    # Schritt 2: Join + Filter auf Time-Window
    merged = df.merge(last_steps, on="vehicle_id", how="left")

    # Schritt 3: Zeitbereich filtern (t - n)
    filtered = merged[
        (merged["time_step"] >= merged["last_time"] - window_size)
        & (merged["time_step"] <= merged["last_time"])
    ]

    # Schritt 4: Letztes Readout pro Fahrzeug und Zeitfenster
    return filtered.drop(columns=["last_time"])

In [110]:
last_readout_window_test = last_readout_window_per_window(readouts_diff_test, window_size=8)
last_readout_window_validation = last_readout_window_per_window(readouts_diff_validation, window_size=8)


NameError: name 'readouts_diff_test' is not defined

In [124]:
display(last_readout_window_validation[["vehicle_id", "time_step", "397_35"]][last_readout_window_validation["vehicle_id"] == 10].sort_values(by="time_step"))

,vehicle_id,time_step,397_35
100,10,377.6,30.0
101,10,383.4,0.0


In [127]:
# combined_window_df_8_test = create_all_fixed_time_index_windows_faster(
#     readouts_df=last_readout_window_test,
#     window_sizes=[8]
# )

combined_window_df_8_validation = create_all_fixed_time_index_windows_faster(
    readouts_df=last_readout_window_validation,
    window_sizes=[8]
)

anzhal an vehicle ids: 5046
Windows sized 8 wird gemacht


Vehicles (w=8):   0%|          | 0/5046 [00:00<?, ?it/s]

anzhal an vehicle ids: 5046


In [117]:
def filter_latest_windows_per_vehicle_and_kind(df: pd.DataFrame) -> pd.DataFrame:
    """
    Behalte nur die Zeilen mit dem maximalen `time_step_current` pro vehicle_id und kind.
    Mehrere Zeilen pro Kombination sind möglich, wenn sie denselben `time_step_current` haben.

    Args:
        df (pd.DataFrame): Long-formatiertes Sliding Window DataFrame mit Spalten:
                           ['vehicle_id', 'kind', 'time_step_current', ...]

    Returns:
        pd.DataFrame: Gefiltertes DataFrame mit nur den letzten Windows pro vehicle_id und kind.
    """
    # Maximalen time_step_current pro vehicle_id + kind bestimmen
    max_time = (
        df.groupby(["vehicle_id", "kind"], sort=False)["time_step_current"]
        .transform("max")
    )

    # Nur Zeilen behalten, bei denen time_step_current == max
    return df[df["time_step_current"] == max_time].copy()


In [ ]:
combined_window_df_8_test = filter_latest_windows_per_vehicle_and_kind(combined_window_df_8_test)
combined_window_df_8_validation = filter_latest_windows_per_vehicle_and_kind(combined_window_df_8_validation)

/tmp/ipykernel_1636/1711113000.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["vehicle_id", "kind"], sort=False)["time_step_current"]


In [ ]:
save_df(df = combined_window_df_8_test, ordner="03_primary", name = "combined_window_df_8_test")
save_df(df = combined_window_df_8_validation, ordner="03_primary", name = "combined_window_df_8_validation")


In [130]:
combined_window_df_8_test = load_df(ordner="03_primary", name = "combined_window_df_8_test")
combined_window_df_8_validation = load_df(ordner="03_primary", name = "combined_window_df_8_validation")

In [156]:
combined_window_df_8_validation[combined_window_df_8_validation["vehicle_id"] == 10]

,id,vehicle_id,time_step,time_step_current,kind,value
0,w8__id10_t383.4,10,377.6,383.4,171_0,1.743750e+05
1,w8__id10_t383.4,10,377.6,383.4,666_0,7.980000e+03
2,w8__id10_t383.4,10,377.6,383.4,427_0,8.090576e+06
3,w8__id10_t383.4,10,377.6,383.4,837_0,2.540000e+03
4,w8__id10_t383.4,10,377.6,383.4,167_0,0.000000e+00
5,w8__id10_t383.4,10,377.6,383.4,167_1,1.368360e+05
6,w8__id10_t383.4,10,377.6,383.4,167_2,3.977270e+05
7,w8__id10_t383.4,10,377.6,383.4,167_3,4.478180e+05
8,w8__id10_t383.4,10,377.6,383.4,167_4,1.427133e+06
9,w8__id10_t383.4,10,377.6,383.4,167_5,6.693596e+06


In [93]:
display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1][combined_window_df_8_test["kind"] == "171_0"].sort_values(by="time_step"))

/tmp/ipykernel_10489/4235577928.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1][combined_window_df_8_test["kind"] == "171_0"].sort_values(by="time_step"))


,id,vehicle_id,time_step,time_step_current,kind,value
210,w8__id1_t224,1,216.8,224.0,171_0,118830.0
315,w8__id1_t224,1,222.6,224.0,171_0,149970.0


### 5. Feature-Extraktion mit tsfresh

In [22]:
def extract_tsfresh_features_for_all_windows(
    df_windows: pd.DataFrame,
    anzahl_workers: int = 8
) -> pd.DataFrame:
    """
    Extrahiert die wichtigsten tsfresh-Features pro Sensor und Window-ID 
    (z. B. w8__id42_t123.4) im Long-Format.
    
    Args:
        df_windows (pd.DataFrame): DataFrame mit den Spalten:
            ['id', 'vehicle_id', 'time_step', 'time_step_current', 'kind', 'value']
        anzahl_workers (int): Anzahl paralleler Prozesse für tsfresh.

    Returns:
        pd.DataFrame: Wide-Format mit reduzierter Featureanzahl + vehicle_id + time_step_current.
    """

    distributor = MultiprocessingDistributor(
        n_workers=anzahl_workers,
        disable_progressbar=False,
        progressbar_title="tsfresh Feature Extraction"
    )

    selected_fc_parameters = {
        "mean": None,
        "median": None,
        "standard_deviation": None,
        "minimum": None,
        "maximum": None
    }

    features_df = extract_features(
        df_windows,
        column_id="id",
        column_sort="time_step",
        column_kind="kind",
        column_value="value",
        default_fc_parameters=selected_fc_parameters,
        impute_function=impute,
        distributor=distributor
    )

    # vehicle_id und time_step_current ergänzen
    id_metadata = df_windows[["id", "vehicle_id", "time_step_current"]].drop_duplicates("id")
    features_df = features_df.merge(id_metadata, how="left", left_index=True, right_on="id")
    features_df = features_df.set_index("id")
    features_df = features_df.rename(columns={"time_step_current": "time_step"})

    float_cols = features_df.select_dtypes(include=["float64"]).columns.difference(["vehicle_id", "time_step"])
    features_df[float_cols] = features_df[float_cols].astype("float32")

    return features_df

#### Extra funktions for chunk execution

In [10]:
def split_and_save_dataframe_by_id(
    df: pd.DataFrame,
    id_column: str,
    output_dir: Union[str, Path],
    n_parts: int = 10,
    file_prefix: str = "chunk"
) -> None:
    """
    Splits a DataFrame into `n_parts` based on unique values in the `id_column`
    and saves each chunk as a separate Parquet file.

    Args:
        df (pd.DataFrame): Input DataFrame in long format.
        id_column (str): Column name used to split the data (e.g., "id").
        output_dir (Union[str, Path]): Directory to save the Parquet chunks.
        n_parts (int, optional): Number of parts to split into. Defaults to 10.
        file_prefix (str, optional): Prefix for output files. Defaults to "chunk".

    Returns:
        None
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    unique_ids = df[id_column].unique()
    total_ids = len(unique_ids)
    chunk_size = total_ids // n_parts

    for i in range(n_parts):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < n_parts - 1 else total_ids
        ids_chunk = unique_ids[start:end]

        df_chunk = df[df[id_column].isin(ids_chunk)].copy()
        output_file = output_dir / f"{file_prefix}_{i+1:02}.parquet"

        df_chunk.to_parquet(output_file, index=False)
        print(f"[{i+1}/{n_parts}] Saved {len(df_chunk)} rows to {output_file}")


In [41]:
split_and_save_dataframe_by_id(
    df=combined_window_df_8_train,
    id_column="id",
    output_dir="../data/03_primary/train_chunks_parquet_pre_tsfresh",
    n_parts=10,
    file_prefix="train"
)

[1/10] Saved 27306090 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_01.parquet
[2/10] Saved 21131145 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_02.parquet
[3/10] Saved 20701485 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_03.parquet
[4/10] Saved 21361095 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_04.parquet
[5/10] Saved 20946345 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_05.parquet
[6/10] Saved 20242635 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_06.parquet
[7/10] Saved 20100780 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_07.parquet
[8/10] Saved 19699995 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_08.parquet
[9/10] Saved 20085660 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_09.parquet
[10/10] Saved 22785840 rows to ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_10.parquet


In [15]:
split_and_save_dataframe_by_id(
    df=combined_window_df_8_test_all,
    id_column="id",
    output_dir="../data/03_primary/test_chunks_parquet_pre_tsfresh",
    n_parts=10,
    file_prefix="test"
)

[1/10] Saved 4850685 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_01.parquet
[2/10] Saved 3778740 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_02.parquet
[3/10] Saved 3838590 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_03.parquet
[4/10] Saved 3688545 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_04.parquet
[5/10] Saved 3657150 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_05.parquet
[6/10] Saved 3697995 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_06.parquet
[7/10] Saved 3585750 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_07.parquet
[8/10] Saved 3343515 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_08.parquet
[9/10] Saved 3431505 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_09.parquet
[10/10] Saved 3972675 rows to ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_10.parquet


In [12]:
def extract_tsfresh_features_from_parquet_chunk(
    input_parquet: Union[str, Path],
    output_parquet: Union[str, Path],
    anzahl_workers: int = 8,
) -> None:
    """
    Loads a long-format window DataFrame from a Parquet file, performs tsfresh feature extraction,
    and writes the resulting wide-format features back to another Parquet file.

    Args:
        input_parquet (Union[str, Path]): Path to input parquet file (long format).
        output_parquet (Union[str, Path]): Path where to save extracted features (wide format).
        anzahl_workers (int, optional): Number of parallel processes for tsfresh. Default is 8.

    Returns:
        None
    """
    input_parquet = Path(input_parquet)
    output_parquet = Path(output_parquet)
    
    print(f"Loading: {input_parquet}")
    df = pd.read_parquet(input_parquet)


        
    distributor = MultiprocessingDistributor(
    n_workers=anzahl_workers,
    disable_progressbar=False,
    progressbar_title="tsfresh Feature Extraction"
        )
    
    selected_fc_parameters = {
        "mean": None,
        "median": None,
        "standard_deviation": None,
        "minimum": None,
        "maximum": None
    }

    features_df = extract_features(
        df,
        column_id="id",
        column_sort="time_step",
        column_kind="kind",
        column_value="value",
        default_fc_parameters=selected_fc_parameters,  # <<< geändert!
        impute_function=impute,
        distributor=distributor
    )

    id_metadata = df[["id", "vehicle_id", "time_step_current"]].drop_duplicates("id")
    features_df = features_df.merge(id_metadata, how="left", left_index=True, right_on="id")
    features_df = features_df.set_index("id")
    features_df = features_df.rename(columns={"time_step_current": "time_step"})

    float_cols = features_df.select_dtypes(include="float64").columns.difference(["vehicle_id", "time_step"])
    features_df[float_cols] = features_df[float_cols].astype("float32")

    print(f"💾 Saving: {output_parquet}")
    features_df.to_parquet(output_parquet)


In [17]:
gc.collect()

1133

In [9]:
# ...existing code...
def batch_extract_tsfresh_features(
    input_dir: Union[str, Path],
    output_dir: Union[str, Path],
    anzahl_workers: int = 5
) -> None:
    """
    Batch-extracts tsfresh features from all Parquet chunks in a directory.

    Args:
        input_dir (Union[str, Path]): Directory containing input Parquet files (long format).
        output_dir (Union[str, Path]): Directory to save output Parquet files (wide format).
        anzahl_workers (int): Number of parallel workers for tsfresh extraction.

    Returns:
        None

    Example:
        batch_extract_tsfresh_features(
            input_dir="../data/03_primary/train_chunks_parquet_pre_tsfresh",
            output_dir="../data/03_primary/train_features_parquet",
            anzahl_workers=5
        )
    """
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    for f in sorted(input_dir.glob("*.parquet")):
        extract_tsfresh_features_from_parquet_chunk(
            input_parquet=f,
            output_parquet=output_dir / f"{f.stem}_features.parquet",
            anzahl_workers=anzahl_workers
        )
        gc.collect()  # Free RAM after each chunk

# ...existing code...

#### Calculate Tsfresh Features

In [ ]:
batch_extract_tsfresh_features(
    input_dir="../data/03_primary/train_chunks_parquet_pre_tsfresh",
    output_dir="../data/03_primary/train_features_parquet",
    anzahl_workers=5
)

Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_07.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:33<00:00, 70.95s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_07_features.parquet
Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_08.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:40<00:00, 71.21s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_08_features.parquet
Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_09.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:48<00:00, 71.55s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_09_features.parquet
Loading: ../data/03_primary/train_chunks_parquet_pre_tsfresh/train_10.parquet


tsfresh Feature Extraction: 100%|██████████| 25/25 [29:51<00:00, 71.67s/it]  


💾 Saving: ../data/03_primary/train_features_parquet/train_10_features.parquet


In [20]:
readout_tsfresh_features_test = extract_tsfresh_features_for_all_windows(combined_window_df_8_test, anzahl_workers=18)


NameError: name 'extract_tsfresh_features_for_all_windows' is not defined

In [13]:
input_dir="../data/03_primary/test_chunks_parquet_pre_tsfresh"
output_dir="../data/03_primary/test_features_parquet"
anzahl_workers=18

input_dir = Path(input_dir)
output_dir = Path(output_dir)
output_dir.mkdir(exist_ok=True)

for f in sorted(input_dir.glob("*.parquet")):
    extract_tsfresh_features_from_parquet_chunk(
        input_parquet=f,
        output_parquet=output_dir / f"{f.stem}_features.parquet",
        anzahl_workers=anzahl_workers
    )
    gc.collect()

Loading: ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_01.parquet


tsfresh Feature Extraction: 100%|██████████| 90/90 [05:01<00:00,  3.35s/it]


💾 Saving: ../data/03_primary/test_features_parquet/test_01_features.parquet
Loading: ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_02.parquet


tsfresh Feature Extraction: 100%|██████████| 90/90 [04:55<00:00,  3.28s/it]


💾 Saving: ../data/03_primary/test_features_parquet/test_02_features.parquet
Loading: ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_03.parquet


...

💾 Saving: ../data/03_primary/test_features_parquet/test_08_features.parquet
Loading: ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_09.parquet


tsfresh Feature Extraction: 100%|██████████| 90/90 [05:04<00:00,  3.38s/it]


💾 Saving: ../data/03_primary/test_features_parquet/test_09_features.parquet
Loading: ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_10.parquet


tsfresh Feature Extraction: 100%|██████████| 90/90 [05:03<00:00,  3.38s/it]


💾 Saving: ../data/03_primary/test_features_parquet/test_10_features.parquet


In [ ]:
batch_extract_tsfresh_features(
    input_dir="../data/03_primary/test_chunks_parquet_pre_tsfresh",
    output_dir="../data/03_primary/test_features_parquet",
    anzahl_workers=18
)

Loading: ../data/03_primary/test_chunks_parquet_pre_tsfresh/test_01.parquet


tsfresh Feature Extraction:  28%|██▊       | 7/25 [30:56<1:34:45, 315.88s/it]

In [134]:
readout_tsfresh_features_validation = extract_tsfresh_features_for_all_windows(combined_window_df_8_validation, anzahl_workers=6)


tsfresh Feature Extraction: 100%|██████████| 30/30 [01:20<00:00,  2.67s/it]


#### Überprüfen ob die Feature Spalten auf float32 statt float64

In [ ]:
def check_float32_loss(series: pd.Series, tolerance: float = 1e-6) -> bool:
    """
    Prüft, ob eine float64-Spalte verlustfrei (bzw. toleranzbasiert) in float32 konvertiert werden kann.

    Args:
        series (pd.Series): Die Spalte, die geprüft werden soll.
        tolerance (float): Erlaubte maximale Abweichung.

    Returns:
        bool: True, wenn float32 möglich ist, False sonst.
    """
    series_float32 = series.astype("float32")
    max_diff = (series - series_float32.astype("float64")).abs().max()
    return max_diff < tolerance

In [ ]:
for col in readout_tsfresh_features_test.select_dtypes(include="float64"):
    if check_float32_loss(readout_tsfresh_features_test[col]):
        continue
    else:
        print(f"{col} lieber float64 behalten.")


#### Überprüfen der tsfresh Daten

In [ ]:
display(readout_tsfresh_features_test.head(5))

In [33]:
display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1000][combined_window_df_8_test["kind"] == "100_0"].sort_values(by="time_step"))

/tmp/ipykernel_1256/2256109858.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(combined_window_df_8_test[combined_window_df_8_test["vehicle_id"] == 1000][combined_window_df_8_test["kind"] == "100_0"].sort_values(by="time_step"))


,id,vehicle_id,time_step,time_step_current,kind,value
27663,w8__id1000_t404.8,1000,398.2,404.8,100_0,68580.0
27768,w8__id1000_t404.8,1000,398.4,404.8,100_0,672.0
27873,w8__id1000_t404.8,1000,403.2,404.8,100_0,192006.0


In [34]:
display(readout_tsfresh_features_test.head(10))

,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

#### Speichern der Tsfresh Daten

In [15]:
def consolidate_parquet_files(input_dir: Union[str, Path], output_file: Union[str, Path]) -> None:
    """Consolidates multiple parquet files into a single parquet file."""
    input_dir = Path(input_dir)
    output_file = Path(output_file)

    dfs = [pd.read_parquet(f) for f in sorted(input_dir.glob("*.parquet"))]
    df_all = pd.concat(dfs, ignore_index=True)

    df_all.to_parquet(output_file, engine="pyarrow", index=False)
    print(f"Gespeichert unter: {output_file}")


In [ ]:
consolidate_parquet_files(input_dir="../data/03_primary/train_features_parquet", output_file="../data/03_primary/readout_tsfresh_features_train.parquet")

In [16]:
consolidate_parquet_files(input_dir="../data/03_primary/test_features_parquet", output_file="../data/03_primary/readout_tsfresh_features_test_all.parquet")

Gespeichert unter: ../data/03_primary/readout_tsfresh_features_test_all.parquet


In [31]:
save_df(df = readout_tsfresh_features_test, ordner="03_primary", name = "readout_tsfresh_features_test")

In [135]:
save_df(df = readout_tsfresh_features_validation, ordner="03_primary", name = "readout_tsfresh_features_validation")

In [ ]:
save_df(df = readout_tsfresh_features_test, ordner="03_primary", name = "readout_tsfresh_features_test_all")

In [136]:
df = load_df(ordner="03_primary", name = "readout_tsfresh_features_validation")

### 6. Label-Erstellung

### Transformation für train labels

In [16]:
label_train = load_specific_raw_data("train_tte")

In [17]:
display(label_train.head())

,vehicle_id,length_of_study_time_step,in_study_repair
0,0,510.0,0
1,2,281.8,0
2,3,293.4,0
3,4,210.0,0
4,5,360.4,0


In [24]:
def prepare_train_labels(df_labels: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare the training labels for survival analysis models.

    Args:
        df_labels (pd.DataFrame): DataFrame with columns ['vehicle_id', 'length_of_study_time_step', 'in_study_repair'].
                                  'in_study_repair' = 1 (event occurred), 0 (censored).

    Returns:
        pd.DataFrame: DataFrame with additional columns for Cox/RSF and AFT.
    """

    # Einheitliche Benennung für Event und Dauer
    df_labels = df_labels.copy()
    df_labels["duration"] = df_labels["length_of_study_time_step"]
    df_labels["event"] = df_labels["in_study_repair"]

    # AFT-Intervalle
    df_labels["upper_bound"] = np.where(df_labels["event"] == 1,
                                        df_labels["duration"],
                                        np.inf)
    return df_labels.drop(columns=["length_of_study_time_step", "in_study_repair"])

In [30]:
label_train_prepare = prepare_train_labels(label_train)

In [31]:
display(label_train_prepare[label_train_prepare["event"]==1].head())

,vehicle_id,duration,event,upper_bound
10,22,190.8,1,190.8
13,27,354.6,1,354.6
15,29,425.2,1,425.2
34,52,235.0,1,235.0
40,61,293.0,1,293.0


### Transformation für test und validation labels

In [137]:
label_test = load_specific_raw_data("test_labels")
label_validation = load_specific_raw_data("validation_labels")

In [33]:
display(label_test.head())

,vehicle_id,class_label
0,1,0
1,6,0
2,7,0
3,9,0
4,11,0


In [138]:
features_test = load_df(ordner="03_primary", name = "readout_tsfresh_features_test")

In [ ]:
##### Tests

In [42]:
subset = features_test[features_test["vehicle_id"].isin([1, 6, 7, 9, 11])]
display(subset)


,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

In [139]:
label_test_timestep = label_test.merge(features_test[["vehicle_id", "time_step"]], on="vehicle_id", how="left")

In [140]:
display(label_test_timestep.head())

,vehicle_id,class_label,time_step
0,1,0,224.0
1,6,0,90.2
2,7,0,424.0
3,9,0,340.0
4,11,0,433.8


In [20]:
def prepare_test_validaten_labels(df_labels: pd.DataFrame) -> pd.DataFrame:
    """
    Konvertiert Klassenlabels (0–4) in duration & event für Survival-Modelle.

    Args:
        test_labels_df (pd.DataFrame): Muss Spalten ['vehicle_id','class_label'] enthalten.

    Returns:
        pd.DataFrame mit Spalten ['vehicle_id', 'duration', 'event', 'upper_bound']
    """

    # Mittlere Restlebensdauer in Zeitschritten pro Klasse
    # Mapping von Klassen zu Intervallen
    class_to_remaining_life = {
        4: (0, 6),     # Klasse 4: 0–6
        3: (6, 12),    # 6–12
        2: (12, 24),   # 12–24
        1: (24, 48),   # 24–48
        0: (48, 48)  # 48
    }

    df = df_labels.copy()

    # Funktion: random Zahl im angegebenen Intervall
    def random_duration(class_label):
        low, high = class_to_remaining_life[class_label]
        if low == high:
            return low   # feste Zahl (48)
        else:
            return np.random.randint(low, high) # ganzzahlig, high ist exklusiv

    df["duration"] = df["class_label"].apply(random_duration)

    # Event-Status: 1 = Defekt, 0 = Zensiert
    df["event"] = np.where(df["class_label"] != 0, 1, 0)

    df["upper_bound"] = np.where(df["event"] == 1,
                                        df["duration"],
                                        np.inf)

    return df


In [141]:
labels_test_prepared = prepare_test_validaten_labels(label_test)
labels_validation_prepared = prepare_test_validaten_labels(label_validation)

##### Tests

In [142]:
display(labels_test_prepared[labels_test_prepared["vehicle_id"].isin([1011,1430,1502,2084,1])].head(10))

,vehicle_id,class_label,duration,event,upper_bound
0,1,0,48,0,inf
151,1011,3,7,1,7.0
203,1430,1,43,1,43.0
215,1502,4,0,1,0.0
314,2084,2,14,1,14.0


In [62]:
subset = features_test[features_test["vehicle_id"].isin([1011,1430,1502,2084,1])]
display(subset)

,100_0__mean,100_0__median,100_0__standard_deviation,100_0__minimum,100_0__maximum,158_0__mean,158_0__median,158_0__standard_deviation,158_0__minimum,158_0__maximum,158_1__mean,158_1__median,158_1__standard_deviation,158_1__minimum,158_1__maximum,158_2__mean,158_2__median,158_2__standard_deviation,158_2__minimum,158_2__maximum,158_3__mean,158_3__median,158_3__standard_deviation,158_3__minimum,158_3__maximum,158_4__mean,158_4__median,158_4__standard_deviation,158_4__minimum,158_4__maximum,158_5__mean,158_5__median,158_5__standard_deviation,158_5__minimum,158_5__maximum,158_6__mean,158_6__median,158_6__standard_deviation,158_6__minimum,158_6__maximum,158_7__mean,158_7__median,158_7__standard_deviation,158_7__minimum,158_7__maximum,158_8__mean,158_8__median,158_8__standard_deviation,158_8__minimum,158_8__maximum,158_9__mean,158_9__median,158_9__standard_deviation,158_9__minimum,158_9__maximum,167_0__mean,167_0__median,167_0__standard_deviation,167_0__minimum,167_0__maximum,167_1__mean,167_1__median,167_1__standard_deviation,167_1__minimum,167_1__maximum,167_2__mean,167_2__median,167_2__standard_deviation,167_2__minimum,167_2__maximum,167_3__mean,167_3__median,167_3__standard_deviation,167_3__minimum,167_3__maximum,167_4__mean,167_4__median,167_4__standard_deviation,167_4__minimum,167_4__maximum,167_5__mean,167_5__median,167_5__standard_deviation,167_5__minimum,167_5__maximum,167_6__mean,167_6__median,167_6__standard_deviation,167_6__minimum,167_6__maximum,167_7__mean,167_7__median,167_7__standard_deviation,167_7__minimum,167_7__maximum,167_8__mean,167_8__median,167_8__standard_deviation,167_8__minimum,167_8__maximum,167_9__mean,167_9__median,167_9__standard_deviation,167_9__minimum,167_9__maximum,171_0__mean,171_0__median,171_0__standard_deviation,171_0__minimum,171_0__maximum,272_0__mean,272_0__median,272_0__standard_deviation,272_0__minimum,272_0__maximum,272_1__mean,272_1__median,272_1__standard_deviation,272_1__minimum,272_1__maximum,272_2__mean,272_2__median,272_2__standard_deviation,272_2__minimum,272_2__maximum,272_3__mean,272_3__median,272_3__standard_deviation,272_3__minimum,272_3__maximum,272_4__mean,272_4__median,272_4__standard_deviation,272_4__minimum,272_4__maximum,272_5__mean,272_5__median,272_5__standard_deviation,272_5__minimum,272_5__maximum,272_6__mean,272_6__median,272_6__standard_deviation,272_6__minimum,272_6__maximum,272_7__mean,272_7__median,272_7__standard_deviation,272_7__minimum,272_7__maximum,272_8__mean,272_8__median,272_8__standard_deviation,272_8__minimum,272_8__maximum,272_9__mean,272_9__median,272_9__standard_deviation,272_9__minimum,272_9__maximum,291_0__mean,291_0__median,291_0__standard_deviation,291_0__minimum,291_0__maximum,291_1__mean,291_1__median,291_1__standard_deviation,291_1__minimum,291_1__maximum,291_10__mean,291_10__median,291_10__standard_deviation,291_10__minimum,291_10__maximum,291_2__mean,291_2__median,291_2__standard_deviation,291_2__minimum,291_2__maximum,291_3__mean,291_3__median,291_3__standard_deviation,291_3__minimum,291_3__maximum,291_4__mean,291_4__median,291_4__standard_deviation,291_4__minimum,291_4__maximum,291_5__mean,291_5__median,291_5__standard_deviation,291_5__minimum,291_5__maximum,291_6__mean,291_6__median,291_6__standard_deviation,291_6__minimum,291_6__maximum,291_7__mean,291_7__median,291_7__standard_deviation,291_7__minimum,291_7__maximum,291_8__mean,291_8__median,291_8__standard_deviation,291_8__minimum,291_8__maximum,291_9__mean,291_9__median,291_9__standard_deviation,291_9__minimum,291_9__maximum,309_0__mean,309_0__median,309_0__standard_deviation,309_0__minimum,309_0__maximum,370_0__mean,370_0__median,370_0__standard_deviation,370_0__minimum,370_0__maximum,397_0__mean,397_0__median,397_0__standard_deviation,397_0__minimum,397_0__maximum,397_1__mean,397_1__median,397_1__standard_deviation,397_1__minimum,397_1__maximum,397_10__mean,397_10__median,397_10__standard_deviation,397_10__minimum,397_10__maximum,397_11__mean,397_11__median,397_11__standard_deviatio

### Ausführen der label transformations

In [143]:
labels_train_prepare = prepare_train_labels(load_specific_raw_data("train_tte"))
labels_test_prepared = prepare_test_validaten_labels(load_specific_raw_data("test_labels"))
labels_validation_prepared = prepare_test_validaten_labels(load_specific_raw_data("validation_labels"))

### Speichern

In [146]:
save_df(df = labels_train_prepare, ordner = "03_primary", name= "labels_train_prepare")
save_df(df = labels_test_prepared, ordner = "03_primary", name= "labels_test_prepared")
save_df(df = labels_validation_prepared, ordner = "03_primary", name= "labels_validation_prepared")

In [46]:
#### Labels mergen für train mit Duration Berechnung

In [7]:
def calc_duration_per_readout(df_readout: pd.DataFrame, df_labels: pd.DataFrame) -> pd.DataFrame:
    """
    Berechnet die Restlebensdauer (duration) pro Readout und merge label.
    duration = TTE - time_step

    Args:
        df_readout (pd.DataFrame): Muss 'vehicle_id' und 'time_step' enthalten.
        df_labels (pd.DataFrame): Muss 'vehicle_id', 'tte', 'event' enthalten.

    Returns:
        pd.DataFrame: Mit Spalten 'duration', 'class', 'event' und 'upper_bound'.
    """
    df = df_readout.merge(df_labels, on="vehicle_id", how="left")

    # Restlebensdauer berechnen
    df["duration"] = df["duration"] - df["time_step"]
    if "upper_bound" in df.columns:
        df["upper_bound"] = df["upper_bound"] - df["time_step"]

    # Klassengrenzen definieren
    bins = [
        (4, (0, 6)),
        (3, (6, 12)),
        (2, (12, 24)),
        (1, (24, 48)),
        (0, (48, float("inf")))
    ]

    def assign_class(x):
        for cls, (low, high) in bins:
            if low <= x < high:
                return cls
        return None

    df["class"] = df["duration"].apply(assign_class)

    # Sonderfall: Zensierung → immer Klasse 0
    df.loc[df["event"] == 0, "class"] = 0

    return df


#### C-Index Test Data

In [ ]:
df_labels_test = load_df(ordner = "03_primary", name= "labels_test_corr_labels")

In [29]:
def calc_duration_per_readout_test_data(df_readout: pd.DataFrame, df_labels: pd.DataFrame) -> pd.DataFrame:
    """
    Berechnet die Restlebensdauer (duration) pro Readout und merge label.
    duration = TTE - time_step

    Args:
        df_readout (pd.DataFrame): Muss 'vehicle_id' und 'time_step' enthalten.
        df_labels (pd.DataFrame): Muss 'vehicle_id', 'tte', 'event' enthalten.

    Returns:
        pd.DataFrame: Mit Spalten 'duration', 'class', 'event' und 'upper_bound'.
    """

    df = df_readout.merge(df_labels, on="vehicle_id", how="left")

    df["time"] = df["duration"] + df["time_step"]

    df["time"] = df.groupby("vehicle_id")["time"].transform("max")

    

    # Restlebensdauer berechnen
    df["duration"] = df["time"] - df["time_step"]

    # Klassengrenzen definieren
    bins = [
        (4, (0, 6)),
        (3, (6, 12)),
        (2, (12, 24)),
        (1, (24, 48)),
        (0, (48, float("inf")))
    ]

    def assign_class(x):
        for cls, (low, high) in bins:
            if low <= x < high:
                return cls
        return None

    df["class"] = df["duration"].apply(assign_class)

    # Sonderfall: Zensierung → immer Klasse 0
    df.loc[df["event"] == 0, "class"] = 0

    df["upper_bound"] = np.where(df["event"] == 1,
                                        df["duration"],
                                        np.inf)

    return df.drop(columns={"time"})


### 7. Feature-Auswahl

#### univariate Cox-Regression

In [8]:
features_train = load_df(ordner="03_primary", name = "readout_tsfresh_features_train")
df_labels = load_df(ordner="03_primary", name = "labels_train_prepare")
features_train_with_labels = calc_duration_per_readout(df_readout=features_train, df_labels=df_labels)


In [29]:
features_train_with_labels.event.value_counts()

event
0    1016560
1     105801
Name: count, dtype: int64

#### cox univariate

In [10]:
def cox_univariate_feature_screen(
    df: pd.DataFrame,
    duration_col: str = "duration",
    event_col: str = "event",
) -> Dict[str, Dict[str, float]]:
    """
    Univariate Cox-Regression für alle numerischen Features.
    Features werden global z-standardisiert (HR bezieht sich auf 1 SD).
    Gibt pro Feature Hazard Ratio (HR) und p-Wert zurück.

    Erwartet: numerische Feature-Spalten + 'duration' + 'event'.

    Args:
        df (pd.DataFrame): DataFrame mit den Daten
        duration_col (str): Name der Spalte mit den Überlebenszeiten
        event_col (str): Name der Spalte mit den Ereignisindikatoren
    
    Return:
        Dict[str, Dict[str, float]]: Ergebnisse der univariaten Cox-Regression
    """
    results: Dict[str, Dict[str, float]] = {}

    # Kandidaten: alle numerischen Spalten außer Labels und IDs
    feature_cols = [c for c in df.columns if c not in (duration_col, event_col, "vehicle_id", "class_label", "upper_bound")]

    # Z-Transformation aller Features auf einmal
    df_std = df.copy()
    df_std[feature_cols] = (df_std[feature_cols] - df_std[feature_cols].mean()) / df_std[feature_cols].std(ddof=0)

    cph = CoxPHFitter()

    for col in tqdm(feature_cols, desc="Cox Univariate Feature Screen", unit="feature"):
        try:
            sub = df_std[[duration_col, event_col, col]]
            cph.fit(sub, duration_col=duration_col, event_col=event_col, show_progress=False)
            s = cph.summary.loc[col]
            hr = float(s["exp(coef)"])
            p = float(s["p"])

            results[col] = {"HR": hr, "p": p}
        except Exception:
            continue

    return results


In [52]:
feature_selection_dict = cox_univariate_feature_screen(features_train_with_labels)

Cox Univariate Feature Screen:   0%|          | 0/527 [00:00<?, ?feature/s]

/usr/local/lib/python3.12/dist-packages/lifelines/fitters/coxph_fitter.py:1679: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/usr/local/lib/python3.12/dist-packages/lifelines/fitters/coxph_fitter.py:1681: RuntimeWarning: overflow encountered in multiply
  phi_x_is = scores[:, None] * X
/usr/local/lib/python3.12/dist-packages/lifelines/fitters/coxph_fitter.py:1733: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/usr/local/lib/python3.12/dist-packages/lifelines/fitters/coxph_fitter.py:1735: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/usr/local/lib/python3.12/dist-packages/lifelines/fitters/coxph_fitter.py:1740: RuntimeWarning: divide by zero encountered in log
  log_lik = log_lik + dot(x_death_sum, beta) + weighted_average * log(denom).sum()
/usr/local/lib/python3.12/dist-packages/lifelines/fitters/coxph_fitter.py:1727: RuntimeWarning: invalid value encountered in multip

In [13]:
feature_selection_dict = {'100_0__mean': {'HR': 0.9820379146953887, 'p': 5.422594085365109e-08},
 '100_0__median': {'HR': 0.9798457269353015, 'p': 1.2833953011309545e-09},
 '100_0__standard_deviation': {'HR': 1.0417729539305356,
  'p': 2.464688975428679e-87},
 '100_0__minimum': {'HR': 0.9617524800408809, 'p': 1.6602343508460242e-28},
 '100_0__maximum': {'HR': 1.0041112559957077, 'p': 0.16918273291417224},
 '158_0__mean': {'HR': 1.0763485347844324, 'p': 2.209266093081598e-227},
 '158_0__median': {'HR': 1.0743940800902638, 'p': 1.9454444446519826e-212},
 '158_0__standard_deviation': {'HR': 1.0652819063537828,
  'p': 2.384885129771809e-224},
 '158_0__minimum': {'HR': 1.0574569751708989, 'p': 1.851585442394989e-114},
 '158_0__maximum': {'HR': 1.0913036731947672, 'p': 0.0},
 '158_1__mean': {'HR': 1.0289090778562953, 'p': 1.632833585149362e-25},
 '158_1__median': {'HR': 1.0258425257446837, 'p': 9.011063767650262e-20},
 '158_1__standard_deviation': {'HR': 1.0755111636935744,
  'p': 1.2385207583436631e-199},
 '158_1__minimum': {'HR': 1.0011824842086663, 'p': 0.7084309678127934},
 '158_1__maximum': {'HR': 1.0517505324550598, 'p': 3.923804414187396e-110},
 '158_2__mean': {'HR': 1.0098473637561491, 'p': 0.0005810499536367316},
 '158_2__median': {'HR': 1.0069673349604715, 'p': 0.017281750186511655},
 '158_2__standard_deviation': {'HR': 1.0637231190780798,
  'p': 3.7263767955818536e-159},
 '158_2__minimum': {'HR': 0.9841617009467349, 'p': 6.837017419235655e-07},
 '158_2__maximum': {'HR': 1.031044443676895, 'p': 4.658376047234724e-44},
 '158_3__mean': {'HR': 1.0252853234831183, 'p': 1.1683974960337573e-94},
 '158_3__median': {'HR': 1.024860037672152, 'p': 3.2250186672717814e-84},
 '158_3__standard_deviation': {'HR': 1.069728965392965,
  'p': 2.371828082315523e-201},
 '158_3__minimum': {'HR': 1.0190496946644143, 'p': 5.309925284889014e-25},
 '158_3__maximum': {'HR': 1.0314305758694426, 'p': 1.5260636716752522e-200},
 '158_4__mean': {'HR': 1.0272930510133207, 'p': 6.701479939491995e-173},
 '158_4__median': {'HR': 1.027058209424905, 'p': 1.1619132196681547e-158},
 '158_4__standard_deviation': {'HR': 1.0770255271638212,
  'p': 2.4441092128935184e-270},
 '158_4__minimum': {'HR': 1.0231151655796569, 'p': 5.520961396078991e-63},
 '158_4__maximum': {'HR': 1.0329749869944724, 'p': 0.0},
 '158_5__mean': {'HR': 1.0564261457690485, 'p': 0.0},
 '158_5__median': {'HR': 1.055724450906018, 'p': 0.0},
 '158_5__standard_deviation': {'HR': 1.0797870927316, 'p': 0.0},
 '158_5__minimum': {'HR': 1.0453554149337678, 'p': 1.0099718421773043e-119},
 '158_5__maximum': {'HR': 1.0690357829935235, 'p': 0.0},
 '158_6__mean': {'HR': 1.0654180684683148, 'p': 0.0},
 '158_6__median': {'HR': 1.064504940550887, 'p': 0.0},
 '158_6__standard_deviation': {'HR': 1.0776521051352395, 'p': 0.0},
 '158_6__minimum': {'HR': 1.0520208696942386, 'p': 5.51610291229866e-141},
 '158_6__maximum': {'HR': 1.0798823937504471, 'p': 0.0},
 '158_7__mean': {'HR': 1.0615500798713189, 'p': 0.0},
 '158_7__median': {'HR': 1.060564936476846, 'p': 7.861711378730093e-286},
 '158_7__standard_deviation': {'HR': 1.078573726575493,
  'p': 2.329432678073667e-290},
 '158_7__minimum': {'HR': 1.047959592773946, 'p': 8.72361244127877e-110},
 '158_7__maximum': {'HR': 1.075758283460554, 'p': 0.0},
 '158_8__mean': {'HR': 1.056459253812916, 'p': 0.0},
 '158_8__median': {'HR': 1.0559663344775148, 'p': 0.0},
 '158_8__standard_deviation': {'HR': 1.0878629554543757, 'p': 0.0},
 '158_8__minimum': {'HR': 1.0494501891733838, 'p': 2.3031710563996012e-302},
 '158_8__maximum': {'HR': 1.0681776952163278, 'p': 0.0},
 '158_9__mean': {'HR': 1.0775484818714411, 'p': 0.0},
 '158_9__median': {'HR': 1.0761019378653356, 'p': 0.0},
 '158_9__standard_deviation': {'HR': 1.0612737658003273, 'p': 0.0},
 '158_9__minimum': {'HR': 1.062935835430542, 'p': 3.279207469771942e-280},
 '158_9__maximum': {'HR': 1.092499936939983, 'p': 0.0},
 '167_0__mean': {'HR': 0.9268181945832474, 'p': 6.179821938805664e-16},
 '167_0__median': {'HR': 0.9213883105265694, 'p': 1.0425269113860865e-17},
 '167_0__standard_deviation': {'HR': 1.0018650957291828,
  'p': 0.32777934884396276},
 '167_0__minimum': {'HR': 0.9134382639683465, 'p': 2.2827286572434006e-21},
 '167_0__maximum': {'HR': 0.9507938637367893, 'p': 3.4693601159051726e-08},
 '167_1__mean': {'HR': 0.922268475155188, 'p': 2.0413470001159093e-103},
 '167_1__median': {'HR': 0.9224689656145743, 'p': 2.066814493213455e-103},
 '167_1__standard_deviation': {'HR': 1.0062661165813815,
  'p': 0.037617218837583795},
 '167_1__minimum': {'HR': 0.9214089750098325, 'p': 1.0264742869208192e-106},
 '167_1__maximum': {'HR': 0.932675975370575, 'p': 4.6643842094795605e-82},
 '167_2__mean': {'HR': 0.9247137740549827, 'p': 4.278984165560115e-93},
 '167_2__median': {'HR': 0.9240500690074456, 'p': 8.946218796243122e-95},
 '167_2__standard_deviation': {'HR': 1.0176930639032475,
  'p': 1.9947722511310504e-09},
 '167_2__minimum': {'HR': 0.9170361070056826, 'p': 1.487008041666212e-110},
 '167_2__maximum': {'HR': 0.9431215319783993, 'p': 2.4875514533261866e-58},
 '167_3__mean': {'HR': 0.9110480418151008, 'p': 1.8400506353863654e-111},
 '167_3__median': {'HR': 0.908803246768798, 'p': 1.3964653980400166e-116},
 '167_3__standard_deviation': {'HR': 1.032026359378844,
  'p': 2.2122765130793888e-29},
 '167_3__minimum': {'HR': 0.8923615275093192, 'p': 1.0398982561478175e-153},
 '167_3__maximum': {'HR': 0.9420342239885926, 'p': 2.320638993245648e-54},
 '167_4__mean': {'HR': 0.9527471283460489, 'p': 2.9666047713199572e-40},
 '167_4__median': {'HR': 0.9493992108003037, 'p': 2.1619432346949533e-45},
 '167_4__standard_deviation': {'HR': 1.047387202584865,
  'p': 1.4494229079803436e-74},
 '167_4__minimum': {'HR': 0.9273976023060871, 'p': 6.95655893871806e-84},
 '167_4__maximum': {'HR': 0.9831996497216153, 'p': 4.4126463043420003e-07},
 '167_5__mean': {'HR': 1.044861592023995, 'p': 0.0},
 '167_5__median': {'HR': 1.044437021487932, 'p': 3.574049358904499e-289},
 '167_5__standard_deviation': {'HR': 1.086675033501353, 'p': 0.0},
 '167_5__minimum': {'HR': 1.038151496072029, 'p': 5.711181960661637e-117},
 '167_5__maximum': {'HR': 1.0533359708043988, 'p': 0.0},
 '167_6__mean': {'HR': 1.0654723072739818, 'p': 2.546150389508597e-229},
 '167_6__median': {'HR': 1.0635432008532706, 'p': 7.349640998494509e-210},
 '167_6__standard_deviation': {'HR': 1.0634059298943537,
  'p': 7.630414879582125e-271},
 '167_6__minimum': {'HR': 1.0490806622304776, 'p': 8.059381006466421e-106},
 '167_6__maximum': {'HR': 1.0808227696262827, 'p': 0.0},
 '167_7__minimum': {'HR': 1.0306630140731552, 'p': 7.506023372133677e-189},
 '167_8__minimum': {'HR': 0.004750253950014536, 'p': 0.0},
 '167_8__maximum': {'HR': 9.229106682844896e-15, 'p': 0.0},
 '171_0__mean': {'HR': 1.04393903474623, 'p': 8.420391450798352e-212},
 '171_0__median': {'HR': 1.043382969808059, 'p': 4.532674066841776e-191},
 '171_0__standard_deviation': {'HR': 1.076205727277645,
  'p': 1.4199091091571904e-261},
 '171_0__minimum': {'HR': 1.034322637495003, 'p': 1.1506453169811845e-62},
 '171_0__maximum': {'HR': 1.053381207052444, 'p': 0.0},
 '272_0__mean': {'HR': 0.948860912905996, 'p': 1.1479972563954833e-47},
 '272_0__median': {'HR': 0.9484439554513866, 'p': 1.7992259866141427e-48},
 '272_0__standard_deviation': {'HR': 1.0176106409274988,
  'p': 2.104657068738724e-09},
 '272_0__minimum': {'HR': 0.9422369462695992, 'p': 2.5417135764708923e-59},
 '272_0__maximum': {'HR': 0.9614075578247547, 'p': 1.527351997765867e-29},
 '272_1__mean': {'HR': 0.9279837362541208, 'p': 3.712521994733876e-71},
 '272_1__median': {'HR': 0.92617854639814, 'p': 1.4732626523724703e-74},
 '272_1__standard_deviation': {'HR': 1.0231654593522226,
  'p': 3.687930251719185e-14},
 '272_1__minimum': {'HR': 0.9156311237355226, 'p': 1.0756402052514353e-94},
 '272_1__maximum': {'HR': 0.9506921172805258, 'p': 1.4776882481952339e-36},
 '272_2__mean': {'HR': 0.988005353441561, 'p': 0.0031576411701905085},
 '272_2__median': {'HR': 0.9832406374441068, 'p': 5.413000178843071e-05},
 '272_2__standard_deviation': {'HR': 1.0427224804849045,
  'p': 1.3355146512367368e-171},
 '272_2__minimum': {'HR': 0.9473652421702229, 'p': 8.87175088505497e-29},
 '272_2__maximum': {'HR': 1.0204137731871323, 'p': 2.1716483644714395e-09},
 '272_3__mean': {'HR': 1.0608973291858919, 'p': 1.5251387376456783e-145},
 '272_3__median': {'HR': 1.0582518470718716, 'p': 6.159619769763482e-130},
 '272_3__standard_deviation': {'HR': 1.0786649800150168, 'p': 0.0},
 '272_3__minimum': {'HR': 1.035902991825561, 'p': 2.738000803263608e-41},
 '272_3__maximum': {'HR': 1.0822726713280064, 'p': 4.156861114525189e-279},
 '272_4__mean': {'HR': 1.019748171196899, 'p': 6.488461295950612e-16},
 '272_4__median': {'HR': 1.0184209265483917, 'p': 2.383684875086838e-13},
 '272_4__standard_deviation': {'HR': 1.045133067619758,
  'p': 6.347266820672309e-67},
 '272_4__minimum': {'HR': 1.006577366370229, 'p': 0.023368660247753584},
 '272_4__maximum': {'HR': 1.0303772903896764, 'p': 3.7170353105047535e-48},
 '272_5__standard_deviation': {'HR': 1.0428539399277523, 'p': 0.0},
 '272_5__minimum': {'HR': 1.044827538422887, 'p': 1.429162350884356e-309},
 '272_5__maximum': {'HR': 1.062449103481771, 'p': 0.0},
 '272_6__mean': {'HR': 0.0006438976306260559, 'p': 0.0},
 '272_6__median': {'HR': 0.00010023238227004143, 'p': 0.0},
 '272_6__standard_deviation': {'HR': 4.4578832118460675e-12, 'p': 0.0},
 '272_7__mean': {'HR': 1.017214804087129, 'p': 2.060101890180734e-25},
 '272_7__median': {'HR': 1.0149523525472024, 'p': 2.70693746029389e-17},
 '272_7__standard_deviation': {'HR': 1.0195419983761789,
  'p': 6.4163891085189856e-49},
 '272_7__minimum': {'HR': 1.0106122875989574, 'p': 9.27980816452651e-08},
 '272_7__maximum': {'HR': 1.0226506158365383, 'p': 3.4886094168970783e-47},
 '272_8__mean': {'HR': 1.0101701707163253, 'p': 6.4781746337844985e-09},
 '272_8__median': {'HR': 1.007703045349615, 'p': 9.395831146820856e-05},
 '272_8__standard_deviation': {'HR': 1.0103025530099723,
  'p': 8.443112281814616e-14},
 '272_8__minimum': {'HR': 1.0045474695846182, 'p': 0.05153168744295414},
 '272_8__maximum': {'HR': 1.0110830814806144, 'p': 5.74820812501068e-14},
 '272_9__mean': {'HR': 1.007562706009815, 'p': 0.001102166829419515},
 '272_9__median': {'HR': 1.0056182357238554, 'p': 0.02243338510580247},
 '272_9__standard_deviation': {'HR': 1.0099037269729634,
  'p': 1.2842982046754604e-05},
 '272_9__minimum': {'HR': 1.0028649928132674, 'p': 0.2880706737346157},
 '272_9__maximum': {'HR': 1.0102810057054366, 'p': 1.8365951542205735e-05},
 '291_0__mean': {'HR': 0.0029316462628761965, 'p': 5.277109066093998e-33},
 '291_0__median': {'HR': 0.002281481779669742, 'p': 6.601359306836039e-36},
 '291_0__standard_deviation': {'HR': 0.9966630646867052,
  'p': 0.728283812800223},
 '291_0__minimum': {'HR': 0.0003757905808544423, 'p': 6.3546494313406834e-55},
 '291_0__maximum': {'HR': 0.0437636665842928, 'p': 1.7411590891737695e-11},
 '291_1__mean': {'HR': 9.78614569042691e-09, 'p': 0.010681756788196843},
 '291_1__median': {'HR': 0.0003008170594437009, 'p': 0.24510233576933038},
 '291_1__standard_deviation': {'HR': 0.989140001349417,
  'p': 0.8501865449061002},
 '291_1__minimum': {'HR': 1.9048855900390012e-08, 'p': 0.010781893743241162},
 '291_1__maximum': {'HR': 7.946888820583463e-08, 'p': 0.03026318699592004},
 '291_10__mean': {'HR': 0.9771912831089375, 'p': 0.7352076906661635},
 '291_10__median': {'HR': 0.9810896875364682, 'p': 0.7457401811671562},
 '291_10__standard_deviation': {'HR': 0.9911822565475401,
  'p': 0.8030052199649793},
 '291_10__minimum': {'HR': 0.9851679507682815, 'p': 0.7633546629902714},
 '291_10__maximum': {'HR': 0.982701643110248, 'p': 0.6992969412844912},
 '291_2__mean': {'HR': 1.3702065428186712e-112, 'p': 0.10539377779717589},
 '291_2__median': {'HR': 3.587635358590878e-38, 'p': 0.574392777529042},
 '291_2__standard_deviation': {'HR': 0.9879261870208429,
  'p': 0.8750787763719727},
 '291_2__minimum': {'HR': 0.9761521925910787, 'p': 0.9169674235078629},
 '291_2__maximum': {'HR': 0.972598460773524, 'p': 0.8810472176408386},
 '291_3__mean': {'HR': 0.8494190215945503, 'p': 1.405393947981225e-105},
 '291_3__median': {'HR': 0.8478366944518221, 'p': 5.858394215982857e-109},
 '291_3__standard_deviation': {'HR': 1.005666464283788,
  'p': 0.0004429603025495354},
 '291_3__minimum': {'HR': 0.8339326590410255, 'p': 1.006813255837816e-129},
 '291_3__maximum': {'HR': 0.8861040866038716, 'p': 3.1058915755117096e-64},
 '291_4__mean': {'HR': 0.36669746416409504, 'p': 1.390983762815546e-175},
 '291_4__median': {'HR': 0.3675013357331591, 'p': 1.4248922490343084e-178},
 '291_4__standard_deviation': {'HR': 0.9995201900288494,
  'p': 0.9208216174562999},
 '291_4__minimum': {'HR': 0.35786369214929575, 'p': 6.00069457160851e-193},
 '291_4__maximum': {'HR': 0.4410261022684113, 'p': 5.7135347928372066e-123},
 '291_5__mean': {'HR': 0.04920538166357369, 'p': 3.367042447271207e-206},
 '291_5__median': {'HR': 0.05066340930157077, 'p': 3.451529324790237e-207},
 '291_5__standard_deviation': {'HR': 0.9954402538431085,
  'p': 0.7183712840748557},
 '291_5__minimum': {'HR': 0.047646495463195354, 'p': 7.01723019562929e-220},
 '291_5__maximum': {'HR': 0.08011930149230603, 'p': 4.620453789656956e-151},
 '291_6__mean': {'HR': 0.0006310995626373054, 'p': 5.892504958826368e-188},
 '291_6__median': {'HR': 0.00064201576483107, 'p': 4.5571238525542133e-191},
 '291_6__standard_deviation': {'HR': 0.9956650316014743,
  'p': 0.721289304853806},
 '291_6__minimum': {'HR': 0.0005315453796507419, 'p': 2.0841292662508024e-205},
 '291_6__maximum': {'HR': 0.0024514500807664083, 'p': 3.134456041037739e-130},
 '291_7__mean': {'HR': 0.01925413634618484, 'p': 1.86977364559991e-09},
 '291_7__median': {'HR': 0.016926551342192474, 'p': 3.9099804648948776e-10},
 '291_7__standard_deviation': {'HR': 0.9958314993742552,
  'p': 0.7216690471854591},
 '291_7__minimum': {'HR': 0.0073384137877151295, 'p': 2.8298077206409592e-14},
 '291_7__maximum': {'HR': 0.12860608203953558, 'p': 0.0019163221458213578},
 '291_8__mean': {'HR': 0.0038166215293447808, 'p': 0.0007901151721560692},
 '291_8__median': {'HR': 0.0005668615800503497, 'p': 7.287214321587744e-06},
 '291_8__standard_deviation': {'HR': 0.9953354041957253,
  'p': 0.7231250714116396},
 '291_8__minimum': {'HR': 0.0021855780658413555, 'p': 0.00016638106800723117},
 '291_8__maximum': {'HR': 0.9842936056014213, 'p': 0.6618336114456116},
 '291_9__mean': {'HR': 0.984248385568538, 'p': 0.6484062313270992},
 '291_9__median': {'HR': 0.9859761091036031, 'p': 0.6602984058547592},
 '291_9__standard_deviation': {'HR': 0.9953049618360816,
  'p': 0.7233213242891758},
 '291_9__minimum': {'HR': 0.9843413792739875, 'p': 0.7822583838384508},
 '291_9__maximum': {'HR': 0.9882187911067163, 'p': 0.5726250038156852},
 '309_0__mean': {'HR': 1.0023769825669182, 'p': 8.829785365674727e-05},
 '309_0__median': {'HR': 1.0023350190514828, 'p': 0.00016196237069588125},
 '309_0__standard_deviation': {'HR': 1.0584846407390602, 'p': 0.0},
 '309_0__minimum': {'HR': 1.0020503053878618, 'p': 0.004014333915210764},
 '309_0__maximum': {'HR': 1.0026950165054824, 'p': 3.1055267082566515e-07},
 '370_0__mean': {'HR': 0.9918881892665103, 'p': 0.012703226515174372},
 '370_0__median': {'HR': 0.9888499181035788, 'p': 0.0007129744478030506},
 '370_0__standard_deviation': {'HR': 1.0437115492123,
  'p': 1.005451645597706e-82},
 '370_0__minimum': {'HR': 0.9706666516204483, 'p': 2.109287685786094e-16},
 '370_0__maximum': {'HR': 1.0117800548487117, 'p': 0.0001043156393912969},
 '397_0__mean': {'HR': 0.7140724276321609, 'p': 3.5517371989370874e-10},
 '397_0__median': {'HR': 0.9782333623954944, 'p': 8.262605531588545e-12},
 '397_0__standard_deviation': {'HR': 0.9983390110023932,
  'p': 0.7192844123423219},
 '397_0__minimum': {'HR': 0.9629454754020679, 'p': 1.718090296077749e-29},
 '397_0__maximum': {'HR': 0.9930612656678128, 'p': 0.8323075154433461},
 '397_1__mean': {'HR': 1.0002070468942912, 'p': 0.9282652917167693},
 '397_1__median': {'HR': 1.0123378724667524, 'p': 3.09937760775425e-05},
 '397_1__standard_deviation': {'HR': 1.0001786328149005,
  'p': 0.9385947584153234},
 '397_1__minimum': {'HR': 0.991024239964379, 'p': 0.004412205989438178},
 '397_1__maximum': {'HR': 1.0000138211527443, 'p': 0.9955316560929349},
 '397_10__mean': {'HR': 0.9931284009360856, 'p': 0.7242121886459467},
 '397_10__median': {'HR': 0.9917745600845173, 'p': 0.7382990261135663},
 '397_10__standard_deviation': {'HR': 0.9934774875130493,
  'p': 0.719667790851664},
 '397_10__minimum': {'HR': 0.999929548282795, 'p': 0.975544813438049},
 '397_10__maximum': {'HR': 0.9937272375350146, 'p': 0.707922163642229},
 '397_11__mean': {'HR': 0.4244521592665167, 'p': 0.6987050932673831},
 '397_11__median': {'HR': 0.49785921228058266, 'p': 0.6917808251787492},
 '397_11__standard_deviation': {'HR': 0.9892819243720256,
  'p': 0.8676749353080052},
 '397_11__minimum': {'HR': 0.9988125997702715, 'p': 0.7682805474704866},
 '397_11__maximum': {'HR': 0.9892080848899019, 'p': 0.873832566810415},
 '397_12__mean': {'HR': 0.9969006563289726, 'p': 0.634782882684522},
 '397_12__median': {'HR': 0.9973613257215298, 'p': 0.6948481352087645},
 '397_12__standard_deviation': {'HR': 0.995824825914358,
  'p': 0.6389169101385457},
 '397_12__minimum': {'HR': 1.0059355674431856, 'p': 3.3537638867560065e-08},
 '397_12__maximum': {'HR': 0.9963631218953076, 'p': 0.6290824893932292},
 '397_13__mean': {'HR': 0.7614115626872208, 'p': 1.5671967952186105e-11},
 '397_13__median': {'HR': 0.8035523601823992, 'p': 2.5105826356265477e-12},
 '397_13__standard_deviation': {'HR': 0.9970615716790139,
  'p': 0.6288222878957758},
 '397_13__minimum': {'HR': 0.9634396909439316, 'p': 2.780070426135715e-20},
 '397_13__maximum': {'HR': 0.7370856084286522, 'p': 0.0008631995825547529},
 '397_14__mean': {'HR': 0.9989201151151913, 'p': 0.760599647726895},
 '397_14__median': {'HR': 1.0000430230170136, 'p': 0.9857676638914589},
 '397_14__standard_deviation': {'HR': 0.9970260772733546,
  'p': 0.6290771466770584},
 '397_14__minimum': {'HR': 1.019114622266705, 'p': 5.1922562882205e-36},
 '397_14__maximum': {'HR': 0.9979180220642948, 'p': 0.6562260198976548},
 '397_15__mean': {'HR': 0.9991541509953122, 'p': 0.7966331122659802},
 '397_15__median': {'HR': 1.000188069031137, 'p': 0.9329022738363845},
 '397_15__standard_deviation': {'HR': 0.9970634087391004,
  'p': 0.6294861997287589},
 '397_15__minimum': {'HR': 1.0115146812396902, 'p': 7.7576772341389e-46},
 '397_15__maximum': {'HR': 0.9980933025151514, 'p': 0.6671141450004636},
 '397_16__mean': {'HR': 0.9972243001407661, 'p': 0.6455804843304636},
 '397_16__median': {'HR': 0.9991121041426246, 'p': 0.8218779222231288},
 '397_16__standard_deviation': {'HR': 0.9956721526255554,
  'p': 0.6420745580570126},
 '397_16__minimum': {'HR': 1.0023830867179677, 'p': 0.015477132171416013},
 '397_16__maximum': {'HR': 0.9964325626841146, 'p': 0.6282319851115649},
 '397_17__mean': {'HR': 0.9989709956387013, 'p': 0.7550135266782869},
 '397_17__median': {'HR': 0.9988995321294916, 'p': 0.7457673534100101},
 '397_17__standard_deviation': {'HR': 0.9993080202403584,
  'p': 0.8235118801777027},
 '397_17__minimum': {'HR': 0.9992396366616192, 'p': 0.8129916507458985},
 '397_17__maximum': {'HR': 0.9990522954741968, 'p': 0.757205618701396},
 '397_18__mean': {'HR': 1.0213542201006764, 'p': 4.3742657418697205e-31},
 '397_18__median': {'HR': 1.020507389979408, 'p': 7.370050906250504e-27},
 '397_18__standard_deviation': {'HR': 1.035120676006874,
  'p': 9.962978509507619e-163},
 '397_18__minimum': {'HR': 1.008046435452414, 'p': 0.00573731933938437},
 '397_18__maximum': {'HR': 1.030140682301141, 'p': 6.056936327091606e-85},
 '397_19__mean': {'HR': 0.9977643798056498, 'p': 0.662282568597591},
 '397_19__median': {'HR': 0.9993662002210624, 'p': 0.8542205351399568},
 '397_19__standard_deviation': {'HR': 0.996681490340313,
  'p': 0.627620762594624},
 '397_19__minimum': {'HR': 1.0080580199579943, 'p': 0.007380499975691148},
 '397_19__maximum': {'HR': 0.9971269931563266, 'p': 0.6296468078536529},
 '397_2__mean': {'HR': 1.000201861729831, 'p': 0.9301854473086739},
 '397_2__median': {'HR': 1.0008224292458072, 'p': 0.6073908967181566},
 '397_2__standard_deviation': {'HR': 0.9985811456904559,
  'p': 0.7094024549846967},
 '397_2__minimum': {'HR': 1.0673840354700352, 'p': 2.8068646282357043e-255},
 '397_2__maximum': {'HR': 0.9992440453842122, 'p': 0.8080481005913748},
 '397_20__mean': {'HR': 1.0441591353984359, 'p': 1.2404916962547777e-249},
 '397_20__median': {'HR': 1.0435988975437607, 'p': 2.5018214044483964e-228},
 '397_20__standard_deviation': {'HR': 1.0649232075947204,
  'p': 5.3097152611199046e-291},
 '397_20__minimum': {'HR': 1.0361572954371407, 'p': 6.750524124137427e-92},
 '397_20__maximum': {'HR': 1.053748956025273, 'p': 0.0},
 '397_21__mean': {'HR': 0.9993535769119845, 'p': 0.8333669774620398},
 '397_21__median': {'HR': 1.0002954835514264, 'p': 0.8886405995772939},
 '397_21__standard_deviation': {'HR': 0.9972208534691616,
  'p': 0.6318603916474403},
 '397_21__minimum': {'HR': 1.0403756310898513, 'p': 2.647248226352199e-148},
 '397_21__maximum': {'HR': 0.9982677806831084, 'p': 0.680223848471013},
 '397_22__mean': {'HR': 0.9921233646549411, 'p': 0.7443899277332646},
 '397_22__median': {'HR': 0.9966373210078534, 'p': 0.8185739349780145},
 '397_22__standard_deviation': {'HR': 0.9916001214554352,
  'p': 0.7944123065743883},
 '397_22__minimum': {'HR': 1.001305069218226, 'p': 0.24257390082349356},
 '397_22__maximum': {'HR': 0.9922213146108949, 'p': 0.7721028647882774},
 '397_23__mean': {'HR': 0.9858946382314902, 'p': 0.8905885569492544},
 '397_23__median': {'HR': 0.994784164888601, 'p': 0.8943351667698284},
 '397_23__standard_deviation': {'HR': 0.9895531067023348,
  'p': 0.8618852812422062},
 '397_23__minimum': {'HR': 0.9998619057511917, 'p': 0.952413972449344},
 '397_23__maximum': {'HR': 0.9896273955344029, 'p': 0.8621558340863245},
 '397_24__mean': {'HR': 1.0005518625756624, 'p': 0.7457440706362861},
 '397_24__median': {'HR': 1.0004884855970626, 'p': 0.7800273975289584},
 '397_24__standard_deviation': {'HR': 1.0008277987161478,
  'p': 0.5521404476494551},
 '397_24__minimum': {'HR': 1.0000726236733906, 'p': 0.9720754619676453},
 '397_24__maximum': {'HR': 1.0008814161462634, 'p': 0.5702385495829636},
 '397_25__mean': {'HR': 1.0026824953169737, 'p': 0.0006624719470664594},
 '397_25__median': {'HR': 1.0026199889227434, 'p': 0.0012102997153338728},
 '397_25__standard_deviation': {'HR': 1.0023503962348295,
  'p': 0.0005901711682771544},
 '397_25__minimum': {'HR': 1.0019232162266183, 'p': 0.044237357446445136},
 '397_25__maximum': {'HR': 1.0031942376054106, 'p': 2.246486403127424e-05},
 '397_26__mean': {'HR': 0.007878865388250856, 'p': 0.0},
 '397_26__median': {'HR': 0.002432690239114701, 'p': 0.0},
 '397_26__minimum': {'HR': 1.2065784327274214e-05, 'p': 0.0},
 '397_26__maximum': {'HR': 0.008345356617421801, 'p': 0.0},
 '397_27__mean': {'HR': 1.0029241728197151, 'p': 1.5719051900590383e-06},
 '397_27__median': {'HR': 1.0028793541849472, 'p': 3.56647301398354e-06},
 '397_27__standard_deviation': {'HR': 1.0023128227559508,
  'p': 0.0003440712859859936},
 '397_27__minimum': {'HR': 1.0023354855605096, 'p': 0.0003475704540784689},
 '397_27__maximum': {'HR': 1.0033791940067174, 'p': 7.800251525621075e-08},
 '397_28__mean': {'HR': 0.9924578289644611, 'p': 0.754440767286578},
 '397_28__median': {'HR': 0.9905871385353431, 'p': 0.771894370627654},
 '397_28__standard_deviation': {'HR': 0.9915885466899792,
  'p': 0.7961572019900456},
 '397_28__minimum': {'HR': 1.0015804105204278, 'p': 0.09538862016888172},
 '397_28__maximum': {'HR': 0.9923948001154672, 'p': 0.7651799952950837},
 '397_29__mean': {'HR': 0.9975734521633447, 'p': 0.6688625893184357},
 '397_29__median': {'HR': 0.9975009427364073, 'p': 0.6680163870628475},
 '397_29__standard_deviation': {'HR': 0.9982815994802554,
  'p': 0.7478240397715613},
 '397_29__minimum': {'HR': 0.9983726094774227, 'p': 0.7491716433861038},
 '397_29__maximum': {'HR': 0.9976999095938263, 'p': 0.6512603923930567},
 '397_3__mean': {'HR': 1.0011181278157308, 'p': 0.48640925344582786},
 '397_3__median': {'HR': 1.0536370150342862, 'p': 0.0},
 '397_3__standard_deviation': {'HR': 0.9991016462683211,
  'p': 0.795849796520643},
 '397_3__minimum': {'HR': 1.0458190745450875, 'p': 7.015306710383508e-225},
 '397_3__maximum': {'HR': 0.9999629635184891, 'p': 0.9882477222323626},
 '397_30__mean': {'HR': 0.999469798695506, 'p': 0.8436292026851124},
 '397_30__median': {'HR': 0.9993834600130351, 'p': 0.8248494460034147},
 '397_30__standard_deviation': {'HR': 0.9998340279278712,
  'p': 0.9436665129553192},
 '397_30__minimum': {'HR': 0.9994432141129425, 'p': 0.8470371914905228},
 '397_30__maximum': {'HR': 0.9996346738666129, 'p': 0.8822132498233036},
 '397_31__mean': {'HR': 1.0035524711402346, 'p': 0.00015730216983336745},
 '397_31__median': {'HR': 1.003456136671312, 'p': 0.00035917476254593235},
 '397_31__standard_deviation': {'HR': 1.0030867723338062,
  'p': 6.0330156421353214e-06},
 '397_31__minimum': {'HR': 1.0023194147852217, 'p': 0.06851949544049704},
 '397_31__maximum': {'HR': 1.0042793889557684, 'p': 4.854215458214172e-07},
 '397_32__mean': {'HR': 0.17081000818474038, 'p': 0.0},
 '397_32__median': {'HR': 0.11155605652650626, 'p': 0.0},
 '397_32__minimum': {'HR': 0.0019006377348037172, 'p': 0.0},
 '397_32__maximum': {'HR': 0.13330733022842275, 'p': 0.0},
 '397_33__mean': {'HR': 1.0178197772715585, 'p': 4.01224942066638e-309},
 '397_33__maximum': {'HR': 1.0195209742246083, 'p': 0.0},
 '397_34__mean': {'HR': 1.071379109155309, 'p': 0.0},
 '397_34__median': {'HR': 1.0699495564043664, 'p': 0.0},
 '397_34__minimum': {'HR': 1.0581986511201662, 'p': 5.979699929574969e-289},
 '397_34__maximum': {'HR': 1.0820717792308294, 'p': 0.0},
 '397_35__mean': {'HR': 1.0335837576710438, 'p': 1.8451783976488148e-225},
 '397_35__median': {'HR': 1.0329113495332383, 'p': 2.0676176869392507e-208},
 '397_35__standard_deviation': {'HR': 1.0339604347487807,
  'p': 2.190242999197942e-214},
 '397_35__minimum': {'HR': 1.0282062481231364, 'p': 3.527335911081357e-122},
 '397_35__maximum': {'HR': 1.0408308676097, 'p': 1.1454309506231065e-307},
 '397_4__mean': {'HR': 0.9988256252876695, 'p': 0.7622730908731223},
 '397_4__median': {'HR': 1.0725718932287025, 'p': 0.0},
 '397_4__standard_deviation': {'HR': 0.9966821506896997,
  'p': 0.6996951854399135},
 '397_4__minimum': {'HR': 1.0616337884033407, 'p': 6.417771476816688e-298},
 '397_4__maximum': {'HR': 0.9976054092647167, 'p': 0.6936723038680117},
 '397_5__mean': {'HR': 1.0525120208115788, 'p': 0.0},
 '397_5__median': {'HR': 1.0515575319590689, 'p': 2.5591351679515564e-299},
 '397_5__standard_deviation': {'HR': 1.042749621665809, 'p': 0.0},
 '397_5__minimum': {'HR': 1.043774424254236, 'p': 1.5986948560914486e-169},
 '397_5__maximum': {'HR': 1.0630122650982317, 'p': 0.0},
 '397_6__mean': {'HR': 0.5213712819262939, 'p': 1.746232074928684e-05},
 '397_6__median': {'HR': 0.5607262196214284, 'p': 1.2519665882083653e-07},
 '397_6__standard_deviation': {'HR': 0.9971374074560162,
  'p': 0.6531374533536292},
 '397_6__minimum': {'HR': 0.9560634873611142, 'p': 2.9821712348057146e-30},
 '397_6__maximum': {'HR': 0.9943723339937662, 'p': 0.6973156084328846},
 '397_7__mean': {'HR': 0.13005149536737381, 'p': 5.836569259375433e-22},
 '397_7__median': {'HR': 0.19366585263268218, 'p': 7.161813322441788e-24},
 '397_7__standard_deviation': {'HR': 0.9951955698519465,
  'p': 0.655661631188494},
 '397_7__minimum': {'HR': 0.9520352823582634, 'p': 6.594863420305692e-40},
 '397_7__maximum': {'HR': 0.11039518691452405, 'p': 4.0576172129277705e-06},
 '397_8__mean': {'HR': 1.0029235596213215, 'p': 0.0005531966836259702},
 '397_8__median': {'HR': 1.0436080926006552, 'p': 6.6764852138117615e-124},
 '397_8__standard_deviation': {'HR': 1.0011404223805207,
  'p': 0.48101282196290895},
 '397_8__minimum': {'HR': 1.0320685788597015, 'p': 8.889140222799118e-42},
 '397_8__maximum': {'HR': 1.001833262673391, 'p': 0.14468645853622172},
 '397_9__mean': {'HR': 0.9969069669447512, 'p': 0.6978105024689985},
 '397_9__median': {'HR': 1.010111451989795, 'p': 2.7828948895984e-15},
 '397_9__standard_deviation': {'HR': 0.9954041883797234,
  'p': 0.7406305867598257},
 '397_9__minimum': {'HR': 1.007247225386337, 'p': 1.85911827524418e-05},
 '397_9__maximum': {'HR': 0.9959079194000153, 'p': 0.7218497890275652},
 '427_0__mean': {'HR': 1.0425776068776111, 'p': 1.9488160583388294e-188},
 '427_0__median': {'HR': 1.0418984831747429, 'p': 6.871090220105974e-167},
 '427_0__standard_deviation': {'HR': 1.077208483516377,
  'p': 2.2810170921822877e-284},
 '427_0__minimum': {'HR': 1.0313779679015387, 'p': 2.5012654193154294e-46},
 '427_0__maximum': {'HR': 1.052479844854762, 'p': 0.0},
 '459_0__mean': {'HR': 0.9971483055288629, 'p': 0.39575556113366184},
 '459_0__median': {'HR': 0.9939550261676021, 'p': 0.07730997280371714},
 '459_0__standard_deviation': {'HR': 1.022696307964864,
  'p': 5.011416137160484e-101},
 '459_0__minimum': {'HR': 0.969662864899149, 'p': 3.2220595742555104e-15},
 '459_0__maximum': {'HR': 1.017741091860349, 'p': 1.8142378628293808e-11},
 '459_1__mean': {'HR': 1.0121366903505744, 'p': 5.987331104295075e-10},
 '459_1__median': {'HR': 1.0110379356134869, 'p': 9.223989518011006e-08},
 '459_1__standard_deviation': {'HR': 1.0278227903481403,
  'p': 1.9127776229028516e-138},
 '459_1__minimum': {'HR': 0.9987989440835098, 'p': 0.7035531065126173},
 '459_1__maximum': {'HR': 1.0199548861820213, 'p': 9.958727356088251e-40},
 '459_10__mean': {'HR': 1.0750664929265819, 'p': 0.0},
 '459_10__median': {'HR': 1.0739909874253024, 'p': 0.0},
 '459_10__standard_deviation': {'HR': 1.0775120897203472, 'p': 0.0},
 '459_10__minimum': {'HR': 1.0609207179951017, 'p': 3.0223696017296137e-220},
 '459_10__maximum': {'HR': 1.09119200824652, 'p': 0.0},
 '459_11__mean': {'HR': 1.0739256010047222, 'p': 0.0},
 '459_11__median': {'HR': 1.0725137274020662, 'p': 0.0},
 '459_11__standard_deviation': {'HR': 1.069407719808508, 'p': 0.0},
 '459_11__minimum': {'HR': 1.0567957878031504, 'p': 1.4674316576678471e-151},
 '459_11__maximum': {'HR': 1.0899135772476776, 'p': 0.0},
 '459_12__mean': {'HR': 1.0711543137083077, 'p': 3.4618763599585726e-306},
 '459_12__median': {'HR': 1.0694517004055548, 'p': 2.920400172550687e-276},
 '459_12__standard_deviation': {'HR': 1.067779925832272, 'p': 0.0},
 '459_12__minimum': {'HR': 1.0511349282975606, 'p': 1.0654197465259846e-103},
 '459_12__maximum': {'HR': 1.087903562915095, 'p': 0.0},
 '459_13__mean': {'HR': 1.0345765708991763, 'p': 2.526617836132978e-72},
 '459_13__median': {'HR': 1.0330957741327205, 'p': 2.6835031447356723e-61},
 '459_13__standard_deviation': {'HR': 1.0618183325617272,
  'p': 3.907459620164573e-190},
 '459_13__minimum': {'HR': 1.0178338209908324, 'p': 1.0400247643861134e-11},
 '459_13__maximum': {'HR': 1.0480095717973745, 'p': 1.1518707584560125e-176},
 '459_14__mean': {'HR': 0.9270486671015675, 'p': 3.1479254956081344e-96},
 '459_14__median': {'HR': 0.9263489158056787, 'p': 8.557496702302707e-98},
 '459_14__standard_deviation': {'HR': 1.0119388090220038,
  'p': 3.5666387993986576e-05},
 '459_14__minimum': {'HR': 0.9175774181481273, 'p': 6.802550392107338e-116},
 '459_14__maximum': {'HR': 0.9452812939766392, 'p': 1.136076988120601e-59},
 '459_15__mean': {'HR': 1.0503457186693665, 'p': 0.0},
 '459_15__median': {'HR': 1.050053831964652, 'p': 0.0},
 '459_15__standard_deviation': {'HR': 1.083788041223562, 'p': 0.0},
 '459_15__minimum': {'HR': 1.045677721693527, 'p': 0.0},
 '459_15__maximum': {'HR': 1.0585680074747077, 'p': 0.0},
 '459_16__mean': {'HR': 1.049157954603586, 'p': 8.248521266487321e-143},
 '459_16__median': {'HR': 1.0484477055870793, 'p': 7.277734091239121e-137},
 '459_16__standard_deviation': {'HR': 1.0440982450921499,
  'p': 1.136197681176579e-95},
 '459_16__minimum': {'HR': 1.0417187633689002, 'p': 4.155592644050818e-93},
 '459_16__maximum': {'HR': 1.0568037017840983, 'p': 9.667202346285693e-176},
 '459_17__mean': {'HR': 0.9589434747679175, 'p': 0.01756887202905683},
 '459_17__median': {'HR': 0.9581012470204598, 'p': 0.018829452733555636},
 '459_17__standard_deviation': {'HR': 0.9861450038123624,
  'p': 0.1522290124050794},
 '459_17__minimum': {'HR': 0.9552342884566216, 'p': 0.031161438545966192},
 '459_17__maximum': {'HR': 0.9662493467535461, 'p': 0.01901252173533877},
 '459_18__mean': {'HR': 0.9692770460515865, 'p': 0.007841881772867832},
 '459_18__median': {'HR': 0.9675791144895592, 'p': 0.007015098886332107},
 '459_18__standard_deviation': {'HR': 0.9985240005384678,
  'p': 0.6984658937346717},
 '459_18__minimum': {'HR': 0.9627997391738677, 'p': 0.006897951514128693},
 '459_18__maximum': {'HR': 0.9767907666191419, 'p': 0.013623789053849466},
 '459_19__mean': {'HR': 0.8441235155150231, 'p': 0.006582428750284548},
 '459_19__median': {'HR': 0.830101312726458, 'p': 0.006816938331177403},
 '459_19__standard_deviation': {'HR': 0.9900340459531856,
  'p': 0.37727818963996473},
 '459_19__minimum': {'HR': 0.8018248938789955, 'p': 0.009932439229863085},
 '459_19__maximum': {'HR': 0.8895325494648324, 'p': 0.0055267080659588495},
 '459_2__mean': {'HR': 1.0143029303209623, 'p': 1.038807680754371e-06},
 '459_2__median': {'HR': 1.011209481080432, 'p': 0.00017099137272807524},
 '459_2__standard_deviation': {'HR': 1.054812550328449,
  'p': 3.0410971080261052e-201},
 '459_2__minimum': {'HR': 0.9872469587801972, 'p': 0.00010492933696734526},
 '459_2__maximum': {'HR': 1.0376707665865605, 'p': 2.9623311237320517e-49},
 '459_3__mean': {'HR': 1.0106511314259419, 'p': 0.00028328784667292395},
 '459_3__median': {'HR': 1.0075254380911323, 'p': 0.011801712513382764},
 '459_3__standard_deviation': {'HR': 1.0567202777955793,
  'p': 9.918247246490225e-175},
 '459_3__minimum': {'HR': 0.9833888755022171, 'p': 3.9956290832698947e-07},
 '459_3__maximum': {'HR': 1.034068733226203, 'p': 1.7649061989649926e-41},
 '459_4__mean': {'HR': 1.0205494899834162, 'p': 4.49642757196579e-15},
 '459_4__median': {'HR': 1.0179235772067083, 'p': 3.5775459390342105e-11},
 '459_4__standard_deviation': {'HR': 1.0610410505652823,
  'p': 8.664328618585906e-208},
 '459_4__minimum': {'HR': 0.9940601197552166, 'p': 0.05997587234535866},
 '459_4__maximum': {'HR': 1.0400902953963345, 'p': 1.9790983808887763e-79},
 '459_5__mean': {'HR': 1.0308866109564252, 'p': 1.5965903593094304e-47},
 '459_5__median': {'HR': 1.0288118660183083, 'p': 4.84938154106981e-38},
 '459_5__standard_deviation': {'HR': 1.0514161303567853, 'p': 0.0},
 '459_5__minimum': {'HR': 1.0075162288956214, 'p': 0.010154877648034666},
 '459_5__maximum': {'HR': 1.046845078659917, 'p': 3.091443747071238e-159},
 '459_6__mean': {'HR': 1.0377556518649182, 'p': 3.0027360768546585e-105},
 '459_6__median': {'HR': 1.036209591911489, 'p': 3.510047965832267e-88},
 '459_6__standard_deviation': {'HR': 1.0592003502327267, 'p': 0.0},
 '459_6__minimum': {'HR': 1.0195918863236693, 'p': 1.918637008024114e-14},
 '459_6__maximum': {'HR': 1.0517681545005115, 'p': 3.151820949624638e-272},
 '459_7__mean': {'HR': 1.0453071218134053, 'p': 3.447444528622455e-185},
 '459_7__median': {'HR': 1.0439669572700263, 'p': 1.9949424241059473e-159},
 '459_7__standard_deviation': {'HR': 1.0675024524187466, 'p': 0.0},
 '459_7__minimum': {'HR': 1.030063067152748, 'p': 4.874177294436226e-40},
 '459_7__maximum': {'HR': 1.0590455351698282, 'p': 0.0},
 '459_8__mean': {'HR': 1.0519953155045985, 'p': 0.0},
 '459_8__median': {'HR': 1.0511840611075711, 'p': 0.0},
 '459_8__standard_deviation': {'HR': 1.061993881350319, 'p': 0.0},
 '459_8__minimum': {'HR': 1.0418125515934595, 'p': 1.718874238745372e-125},
 '459_8__maximum': {'HR': 1.0639633050867183, 'p': 0.0},
 '459_9__mean': {'HR': 1.0641885786816523, 'p': 0.0},
 '459_9__median': {'HR': 1.0635579826872068, 'p': 0.0},
 '459_9__standard_deviation': {'HR': 1.0818658861912505, 'p': 0.0},
 '459_9__minimum': {'HR': 1.0552723645102624, 'p': 3.690933332992682e-289},
 '459_9__maximum': {'HR': 1.077531652201068, 'p': 0.0},
 '666_0__mean': {'HR': 0.005530683508048433, 'p': 0.0},
 '666_0__standard_deviation': {'HR': 1.0780260104607595, 'p': 0.0},
 '666_0__maximum': {'HR': 9.820381742568782e-08, 'p': 0.0},
 '835_0__mean': {'HR': 1.0327720784512402, 'p': 2.025586213138259e-87},
 '835_0__median': {'HR': 1.0317336535394388, 'p': 2.3712151707929584e-73},
 '835_0__standard_deviation': {'HR': 1.0781100564137425,
  'p': 5.859659566788304e-250},
 '835_0__minimum': {'HR': 1.0168992057818629, 'p': 2.0883472787779144e-10},
 '835_0__maximum': {'HR': 1.0432183596316524, 'p': 5.685762456143649e-246},
 '837_0__standard_deviation': {'HR': 0.18594095988440776, 'p': 0.0},
 'time_step': {'HR': 1.4846967657081886, 'p': 0.0},
 'class': {'HR': 2445.669711158574, 'p': 0.5659362760184982}}

In [15]:
display(len(feature_selection_dict))

499

In [12]:
# Speichern
with open("../data/03_primary/feature_selection_dict_all.json", "w") as f:
    json.dump(feature_selection_dict, f, indent=2)

In [13]:
# Laden
with open("../data/03_primary/feature_selection_dict_all.json", "r") as f:
    feature_selection_dict = json.load(f)


In [32]:
cox_p_werte = pd.DataFrame(feature_selection_dict).T.sort_values("p").head()

In [33]:
display(cox_p_werte)

,HR,p
459_15__median,1.050054,0.0
459_12__standard_deviation,1.067780,0.0
459_11__maximum,1.089914,0.0
459_11__standard_deviation,1.069408,0.0
459_11__median,1.072514,0.0


In [34]:
def auswahl_features_p(feature_selection_dict: dict, p_threshold: float) -> dict:
    """
    Select features based on p-value threshold.

    Args:
        feature_selection_dict (dict): Dictionary containing feature names as keys and their p-values as values.
        p_threshold (float): The p-value threshold for feature selection.

    Returns:
        dict: A dictionary containing the selected features.
    """
    
    selected_features = {}
    for key, value in feature_selection_dict.items():
        if value['p'] <= p_threshold:
            selected_features[key] = value
    return selected_features

In [ ]:
selected_features_p = auswahl_features_p(feature_selection_dict, p_threshold=0.01)

print(f"Original feature count: {features_train.shape[1]-1}, Selected feature count: {len(selected_features_p)}")

Original feature count: 499, Selected feature count: 351


### Pearsonkorrelationsmatrix

In [148]:
features_train = load_df(ordner="03_primary", name = "readout_tsfresh_features_train")


KeyboardInterrupt: 

In [180]:
def drop_correlated_features_keep_best_p(
    results_dict: dict,
    df: pd.DataFrame,
    threshold: float = 0.5
) -> dict:
    """
    Wähle Features so, dass keine beiden |rho| >= threshold haben.
    Behalte jeweils das Feature mit dem kleineren p (höchster Evidenz).
    Deterministisch durch Sortierung nach p.
    """
    # Sicherstellen, dass alle Features im df vorhanden & numerisch sind
    feats = [f for f in results_dict.keys() if f in df.columns and f not in ["time_step"]]
    if len(feats) == 0:
        return {}
    
    # # Filter nach Anteil Nicht-Null-Werten
    # nonzero_ratio = (df[feats] != 0).mean()
    # feats = [f for f in feats if nonzero_ratio[f] >= 0.6]
    # if len(feats) == 0:
    #     return {}
    
    #  Korrelationen auf genau diesen Spalten berechnen
    corr = df[feats].corr(method="pearson").abs()


    #  Features nach p und hr aufsteigend sortieren (bestes zuerst)
    feats_sorted = sorted(
    feats,
    key=lambda f: (
        results_dict[f]["p"],
        -abs(1 - results_dict[f]["HR"])   # Minus für absteigend
                        )
                )
    

    selected = ["time_step"]
    blocked = set()
    
    for f in feats_sorted:
        if f in blocked:
            continue
        # f behalten
        selected.append(f)
        # alle stark korrelierten blocken (inkl. symmetrisch)
        high_corr_partners = corr.index[(corr[f] >= threshold) & (corr.index != f)].tolist()
        
        blocked.update(high_corr_partners)
        # Hinweis: f bleibt natürlich erhalten

    # Ergebnisse filtern
    return {f: results_dict[f] for f in selected}

In [ ]:
selected_features_p_corr = drop_correlated_features_keep_best_p(selected_features_p, features_train, threshold=0.5)
print(f"Original feature count: {features_train.shape[1]-1}, Selected feature count: {len(selected_features_p)}, Selected feature count after correlation removal with 0.5: {len(selected_features_p_corr)}")

Original feature count: 499, Selected feature count: 351, Selected feature count after correlation removal with 0.5: 39


In [176]:
display(selected_features_p_corr)

{'time_step': {'HR': 1.4846967657081886, 'p': 0.0},
 '666_0__maximum': {'HR': 9.820381742568782e-08, 'p': 0.0},
 '397_26__minimum': {'HR': 1.2065784327274214e-05, 'p': 0.0},
 '158_0__maximum': {'HR': 1.0913036731947672, 'p': 0.0},
 '397_34__maximum': {'HR': 1.0820717792308294, 'p': 0.0},
 '167_6__maximum': {'HR': 1.0808227696262827, 'p': 0.0},
 '397_4__median': {'HR': 1.0725718932287025, 'p': 0.0},
 '459_7__maximum': {'HR': 1.0590455351698282, 'p': 0.0},
 '272_5__minimum': {'HR': 1.044827538422887, 'p': 1.429162350884356e-309},
 '158_9__minimum': {'HR': 1.062935835430542, 'p': 3.279207469771942e-280},
 '291_5__minimum': {'HR': 0.047646495463195354, 'p': 7.01723019562929e-220},
 '459_16__maximum': {'HR': 1.0568037017840983, 'p': 9.667202346285693e-176},
 '167_3__minimum': {'HR': 0.8923615275093192, 'p': 1.0398982561478175e-153},
 '397_21__minimum': {'HR': 1.0403756310898513, 'p': 2.647248226352199e-148},
 '397_18__maximum': {'HR': 1.030140682301141, 'p': 6.056936327091606e-85},
 '167_1_

In [17]:
selected_features_p_corr = {'time_step': {'HR': 1.4846967657081886, 'p': 0.0},
 '666_0__maximum': {'HR': 9.820381742568782e-08, 'p': 0.0},
 '397_26__minimum': {'HR': 1.2065784327274214e-05, 'p': 0.0},
 '158_0__maximum': {'HR': 1.0913036731947672, 'p': 0.0},
 '397_34__maximum': {'HR': 1.0820717792308294, 'p': 0.0},
 '167_6__maximum': {'HR': 1.0808227696262827, 'p': 0.0},
 '397_4__median': {'HR': 1.0725718932287025, 'p': 0.0},
 '459_7__maximum': {'HR': 1.0590455351698282, 'p': 0.0},
 '272_5__minimum': {'HR': 1.044827538422887, 'p': 1.429162350884356e-309},
 '158_9__minimum': {'HR': 1.062935835430542, 'p': 3.279207469771942e-280},
 '291_5__minimum': {'HR': 0.047646495463195354, 'p': 7.01723019562929e-220},
 '459_16__maximum': {'HR': 1.0568037017840983, 'p': 9.667202346285693e-176},
 '167_3__minimum': {'HR': 0.8923615275093192, 'p': 1.0398982561478175e-153},
 '397_21__minimum': {'HR': 1.0403756310898513, 'p': 2.647248226352199e-148},
 '397_18__maximum': {'HR': 1.030140682301141, 'p': 6.056936327091606e-85},
 '167_1__maximum': {'HR': 0.932675975370575, 'p': 4.6643842094795605e-82},
 '459_14__maximum': {'HR': 0.9452812939766392, 'p': 1.136076988120601e-59},
 '397_7__median': {'HR': 0.19366585263268218, 'p': 7.161813322441788e-24},
 '397_0__mean': {'HR': 0.7140724276321609, 'p': 3.5517371989370874e-10},
 '272_2__maximum': {'HR': 1.0204137731871323, 'p': 2.1716483644714395e-09},
 '397_12__minimum': {'HR': 1.0059355674431856, 'p': 3.3537638867560065e-08},
 '397_27__maximum': {'HR': 1.0033791940067174, 'p': 7.800251525621075e-08}}

### relevante Featues selektieren

In [18]:
def select_relevant_features(df_readouts: pd.DataFrame, selected_features: dict) -> pd.DataFrame:
    """
    Reduziert DataFrame auf relevante Features (nach Feature Selection) + Labels.
    
    Args:
        df (pd.DataFrame): Ursprüngliches DataFrame mit allen Features und Labels.
        selected_features (dict): Dict der Form {feature: {"HR":..., "p":..., "p_ph":...}}
                                  aus der Feature Selection.

    Returns:
        pd.DataFrame: DataFrame mit ausgewählten Features.
    """
    # Relevante Spalten aus dem Dict
    feature_cols = list(selected_features.keys())
    feature_cols.extend(["vehicle_id"])

    df_readouts = df_readouts[feature_cols]

    return df_readouts


In [183]:
feature_train_corr = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_train"), selected_features_p_corr)

feature_test_corr = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_test"), selected_features_p_corr)

feature_validation_corr = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_validation"), selected_features_p_corr)

In [19]:
feature_test_corr_all = select_relevant_features(load_df(ordner="03_primary", name = "readout_tsfresh_features_test_all"), selected_features_p_corr)

In [24]:
display(feature_test_corr_all[feature_test_corr_all["vehicle_id"]==1000].sort_values("time_step")[["time_step", "vehicle_id"]])

,time_step,vehicle_id
57,6.4,1000
56,47.6,1000
58,68.0,1000
0,100.8,1000
1,103.0,1000
2,111.2,1000
3,140.6,1000
4,150.0,1000
5,177.2,1000
6,182.4,1000


In [26]:
df_labels=load_df(ordner="03_primary", name = "labels_test_prepared")


### merge mit labels




In [184]:

feature_train_corr_labels = calc_duration_per_readout(df_readout=feature_train_corr, df_labels=load_df(ordner="03_primary", name = "labels_train_prepare"))

feature_validation_corr_labels = pd.merge(feature_validation_corr, load_df(ordner="03_primary", name = "labels_validation_prepared"), on="vehicle_id", how="left")

feature_test_corr_labels = pd.merge(feature_test_corr, load_df(ordner="03_primary", name = "labels_test_prepared"), on="vehicle_id", how="left")


In [30]:
feature_test_corr_labels_all = calc_duration_per_readout_test_data(df_readout=feature_test_corr_all, df_labels=load_df(ordner="03_primary", name = "labels_test_prepared"))

In [31]:
df = load_df(ordner="02_intermediate", name = "readouts_diff_train")

In [186]:
# Für jeden Wert die Bedingung prüfen (==0), mitteln und in %
percent_zeros = (df == 0).mean() * 100
percent_zeros = percent_zeros.round(2)  # 2 Nachkommastellen
percent_zeros_df = pd.DataFrame({
    "Feature": percent_zeros.index,
    "Percent_Zeros": percent_zeros.values
})

print(percent_zeros_df)

        Feature  Percent_Zeros
0    vehicle_id           0.02
1     time_step           0.17
2         171_0           1.57
3         666_0           5.36
4         427_0           1.12
5         837_0          16.60
6         167_0          78.56
7         167_1           1.40
8         167_2           2.34
9         167_3           3.42
10        167_4           3.56
11        167_5           4.19
12        167_6          11.81
13        167_7          36.47
14        167_8          69.87
15        167_9          87.02
16        309_0          48.83
17        272_0           0.80
18        272_1           2.23
19        272_2           2.77
20        272_3           2.86
21        272_4           3.97
22        272_5          14.09
23        272_6          64.23
24        272_7          97.69
25        272_8          99.03
26        272_9          99.26
27        835_0           0.45
28        370_0          70.71
29        291_0           2.28
30        291_1           2.01
31      

In [187]:
# Für jeden Wert die Bedingung prüfen (==0), mitteln und in %
percent_zeros = (feature_train_corr_labels == 0).mean() * 100

print(percent_zeros)

time_step                      0.172137
167_8__maximum                60.902686
666_0__maximum                 2.341047
397_26__minimum                2.607984
272_6__median                 59.249831
837_0__standard_deviation     59.392744
158_0__maximum                 0.811860
459_15__standard_deviation    56.210257
397_34__maximum                1.409440
167_6__maximum                 6.108462
272_3__standard_deviation     55.229556
397_4__median                  0.434798
459_7__maximum                 1.750595
272_5__standard_deviation     58.322590
397_5__standard_deviation     56.284030
272_5__minimum                17.374267
158_9__minimum                24.718428
291_5__minimum                 7.793660
397_35__standard_deviation    66.662865
459_16__maximum               12.564139
272_2__standard_deviation     55.225814
397_18__standard_deviation    55.201847
167_3__minimum                 5.138721
397_21__minimum                2.608875
397_35__minimum               51.154130


In [188]:
display(feature_validation_corr_labels[feature_validation_corr_labels["vehicle_id"]==10].head())

,time_step,167_8__maximum,666_0__maximum,397_26__minimum,272_6__median,837_0__standard_deviation,158_0__maximum,459_15__standard_deviation,397_34__maximum,167_6__maximum,272_3__standard_deviation,397_4__median,459_7__maximum,272_5__standard_deviation,397_5__standard_deviation,272_5__minimum,158_9__minimum,291_5__minimum,397_35__standard_deviation,459_16__maximum,272_2__standard_deviation,397_18__standard_deviation,167_3__minimum,397_21__minimum,397_35__minimum,167_1__maximum,459_14__maximum,272_7__standard_deviation,397_18__mean,397_7__median,167_0__minimum,370_0__minimum,397_0__mean,397_12__minimum,397_27__maximum,272_9__standard_deviation,272_8__median,291_3__standard_deviation,459_19__maximum,vehicle_id,class_label,duration,event,upper_bound
180,383.4,0.0,7980.0,296145.0,0.0,0.0,63892.0,0.0,8295.0,262948.0,0.0,50716.0,391.700012,0.0,0.0,9324.0,124236.0,592.0,0.0,11.6,0.0,0.0,447818.0,27240.0,30.0,136836.0,9778.400391,0.0,46500.0,765421.0,0.0,0.0,1248165.0,50446.0,26700.0,0.0,0.0,0.0,0.0,10,0,48,0,inf


In [189]:
display(feature_train_corr.describe())

,time_step,167_8__maximum,666_0__maximum,397_26__minimum,272_6__median,837_0__standard_deviation,158_0__maximum,459_15__standard_deviation,397_34__maximum,167_6__maximum,272_3__standard_deviation,397_4__median,459_7__maximum,272_5__standard_deviation,397_5__standard_deviation,272_5__minimum,158_9__minimum,291_5__minimum,397_35__standard_deviation,459_16__maximum,272_2__standard_deviation,397_18__standard_deviation,167_3__minimum,397_21__minimum,397_35__minimum,167_1__maximum,459_14__maximum,272_7__standard_deviation,397_18__mean,397_7__median,167_0__minimum,370_0__minimum,397_0__mean,397_12__minimum,397_27__maximum,272_9__standard_deviation,272_8__median,291_3__standard_deviation,459_19__maximum,vehicle_id
count,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06,1.122361e+06
mean,1.426380e+02,6.212423e+03,5.269366e+03,4.582588e+04,2.133349e+03,3.158953e+02,1.310236e+05,2.464600e+02,7.235563e+03,6.878541e+05,2.507086e+05,1.114215e+04,1.108719e+02,4.482464e+04,4.369387e+02,2.116696e+05,6.984772e+04,1.496590e+02,2.230508e+01,1.737879e+02,5.105705e+04,1.424801e+03,4.158597e+05,5.840622e+03,7.327824e+01,2.921282e+05,1.178768e+03,4.801941e+01,8.210615e+03,1.351932e+05,3.565296e+02,1.960412e+05,3.846742e+05,2.676529e+04,6.940706e+03,8.776024e+00,1.740203e+01,1.185333e+02,6.499397e-02,1.563527e+04
std,9.529316e+01,5.172562e+04,8.546644e+03,7.269352e+04,2.351298e+04,1.052648e+03,1.899636e+05,6.286420e+02,1.858838e+04,1.123070e+06,9.086805e+05,1.954777e+04,1.261974e+02,2.005223e+05,1.845654e+03,6.189708e+05,1.937344e+05,4.982033e+03,1.768875e+02,5.594784e+02,2.107766e+05,5.515418e+03,7.033168e+05,1.782117e+04,5.421201e+02,3.256597e+05,1.593791e+03,1.902403e+03,1.795352e+04,8.099407e+06,5.473257e+03,8.023962e+05,8.051030e+06,6.956080e+04,6.818812e+04,4.809545e+02,7.635332e+02,5.825631e+02,5.061148e+00,9.801686e+03
min,0.000000e+00,-2.565750e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.940648e+06,0.000000e+00,0.000000e+00,-3.942000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.090000e+03,0.000000e+00,-2.724400e+03,0.000000e+00,0.000000e+00,-1.616032e+06,0.000000e+00,0.000000e+00,-1.168320e+06,-3.819200e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.640000e+01,0.000000e+00,1.456000e+03,6.917000e+03,0.000000e+00,0.000000e+00,1.584000e+04,0.000000e+00,7.360000e+02,7.011000e+04,0.000000e+00,1.868000e+03,4.810000e+01,0.000000e+00,0.000000e+00,1.183000e+03,1.000000e+00,2.100000e+01,0.000000e+00,3.000000e-01,0.000000e+00,0.000000e+00,6.679600e+04,7.000000e+02,0.000000e+00,1.123810e+05,3.489000e+02,0.000000e+00,3.045250e+03,4.332900e+04,0.000000e+00,0.000000e+00,1.265680e+05,3.972000e+03,2.036000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.728000e+03
50%,1.278000e+02,0.000000e+00,3.430000e+03,2.653200e+04,0.000000e+00,0.000000e+00,6.988800e+04,0.000000e+00,2.232000e+03,3.404550e+05,0.000000e+00,5.353000e+03,8.070000e+01,0.000000e+00,0.000000e+00,3.788200e+04,6.360000e+03,7.800000e+01,0.000000e+00,1.800000e+00,0.000000e+00,0.000000e+00,2.466450e+05,2.408000e+03,0.000000e+00,2.165550e+05,7.241000e+02,0.000000e+00,5.368000e+03,8.389600e+04,0.000000e+00,0.000000e+00,2.562840e+05,1.219200e+04,3.896000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.578300e+04
75%,2.060000e+02,2.025000e+03,6.601000e+03,6.317200e+04,2.470000e+02,1.360000e+02,1.790520e+05,2.217818e+02,6.244000e+03,8.658900

In [190]:
save_df(df = feature_train_corr_labels, ordner="04_feature", name = "feature_train_corr_labels")

save_df(df = feature_test_corr_labels, ordner="04_feature", name = "feature_test_corr_labels")

save_df(df = feature_validation_corr_labels, ordner="04_feature", name = "feature_validation_corr_labels")


In [42]:
save_df(df = feature_test_corr_labels_all, ordner="04_feature", name = "feature_test_corr_labels_all")

In [4]:
df = load_df(ordner="03_primary", name="readout_tsfresh_features_train")